In [77]:
#DONE Initial Solution Finder

import DelimitedFiles
import HiGHS
import JuMP
import Random
import Statistics

using DelimitedFiles
using JuMP
using Random
using Statistics

open(joinpath(@__DIR__, "transp.txt"), "w") do io
    print(
        io,
        """
             . 広瀬中	仙台青	蒲町小	長町中	七郷小	南光台	新田小	仙台高	錦ケ丘	上杉山	宮城野	仙台商	岩切小	榴岡小	東長町	富沢小	富沢中	TSSW	TSDA	TSMP	TSTP	TSMN	TSHW	TSMS	TSH1	TSH2	TSH3  EVACUEES
          TTWR .	.	.	2.85	.	.	.	4.80	.	1.65	2.31	.	.	1.47	4.33	.	4.38	1.39	4.37	3.94	1.99	.	3.88	2.46	3.76	4.34	4.49	894
          ZUHO .	.	.	2.30	.	.	.	4.58	.	2.36	3.37	.	.	2.54	4.12	4.54	3.57	1.20	.	3.00	3.03	.	4.03	3.44	3.85	4.53	4.64	60
          OSAK .	2.05	.	.	.	.	.	1.46	.	2.65	.	.	.	4.45	.	.	.	2.06	4.17	.	.	.	.	.	.	.	.	100000
          TDAI .	.	.	2.54	.	.	.	4.82	.	1.93	2.61	.	.	1.78	4.11	4.98	4.03	1.35	4.69	3.57	2.27	.	3.77	2.70	3.63	4.25	4.39	5000
          AERT .	.	.	3.56	.	4.54	4.64	4.84	.	1.20	1.77	.	.	0.93	4.86	.	.	1.75	3.70	4.72	1.50	.	4.22	2.06	4.13	4.64	4.81	584
          TEPC .	4.81	.	3.86	.	4.33	4.78	4.47	.	0.78	2.04	.	.	1.25	.	.	.	1.58	3.36	4.92	1.82	.	4.64	2.40	4.54	.	.	12500
          SIPO .	.	.	.	.	.	.	.	.	.	.	3.68	.	.	.	.	.	.	.	.	.	4.29	.	.	.	.	.	1653
          MEDI .	3.88	.	3.89	.	4.99	.	3.45	.	0.99	3.21	.	.	2.39	.	.	.	0.57	3.65	4.54	2.97	.	.	3.53	.	.	.	346
          SSTA .	.	4.83	3.35	.	4.71	4.60	.	.	1.47	1.66	.	.	0.81	4.61	.	4.96	1.87	3.94	4.58	1.36	.	3.95	1.90	3.86	4.37	4.54	75608
          AQUA .	.	.	.	.	.	4.97	.	.	.	.	.	4.19	.	.	.	.	.	.	.	.	.	.	.	.	.	.	913
          MOPS .	.	.	.	.	.	.	.	.	.	.	.	5.09	.	.	.	.	.	.	.	.	.	.	.	.	.	.	10000
          YURT .	.	.	.	.	2.70	.	.	.	.	.	1.03	.	.	.	.	.	.	2.58	.	.	0.49	.	.	.	.	.	11467
          CAST .	4.51	.	2.90	.	.	.	3.87	.	2.68	4.27	.	.	3.42	4.85	4.72	3.77	1.15	.	3.01	3.95	.	4.95	4.41	4.76	.	.	87
          RAKU .	.	2.96	3.98	4.13	4.78	3.19	.	.	3.13	0.68	.	.	1.21	4.39	.	.	3.79	4.74	.	0.67	.	3.24	0.13	3.27	3.48	3.68	27000
          TLSW 9.36	4.01	6.62	3.51	7.86	5.56	6.33	3.49	9.50	1.55	3.51	7.02	8.45	2.66	5.30	5.65	4.69	0.00	4.21	4.02	3.24	6.61	5.10	3.77	4.95	5.59	5.72   999999
          TLDA .	4.38	7.45	7.22	8.38	1.86	4.90	4.57	.	2.80	4.08	3.21	5.62	3.91	8.53	9.66	8.71	4.23	0.00	8.16	4.17	2.66	7.74	4.63	7.69	8.09	8.28   999999
          TLMP .	7.36	7.04	1.70	8.21	9.21	8.68	6.67	9.68	5.35	5.76	.	.	5.11	3.11	1.75	0.89	4.02	8.13	0.00	5.41	.	4.03	5.60	3.80	4.45	4.45   999999
          TLTP .	6.56	3.63	3.96	4.79	4.38	3.37	6.26	.	2.47	0.36	7.35	5.98	0.58	4.66	6.50	5.66	3.23	4.16	5.47	0.00	6.78	3.64	0.58	3.64	3.95	4.14   999999
          TLMN .	5.20	9.87	9.82	.	3.07	6.79	5.70	.	5.35	6.67	0.62	6.40	6.57	.	.	.	6.62	2.66	.	6.80	0.00	.	7.22	.	.	.   999999
          TLHW .	9.02	3.23	2.47	4.29	8.01	5.98	8.50	.	5.38	3.86	.	8.86	3.81	1.38	4.04	3.62	5.05	7.72	4.03	3.64	.	0.00	3.37	0.23	0.50	0.62   999999
          TLMS .	7.13	3.05	4.07	4.20	4.65	3.12	6.83	.	3.05	0.55	7.77	5.86	1.14	4.51	6.56	5.77	3.76	4.61	5.66	0.58	7.19	3.37	0.00	3.40	3.62	3.81   999999
          TLH1 .	8.88	3.43	2.24	4.50	8.02	6.09	8.35	.	5.29	3.87	.	8.96	3.78	1.23	3.84	3.40	4.90	7.68	3.80	3.64	.	0.23	3.40	0.00	0.71	0.79   999999
          TLH2 .	9.49	3.02	2.94	3.99	8.27	6.04	8.98	.	5.80	4.13	.	8.92	4.17	1.59	4.32	3.98	5.53	8.07	4.45	3.95	.	0.50	3.62	0.71	0.00	0.20   999999
          TLH3 .	9.63	3.16	2.97	4.09	8.46	6.23	9.11	.	5.97	4.33	.	9.10	4.35	1.50	4.23	3.94	5.66	8.25	4.44	4.14	.	0.62	3.81	0.79	0.20	0.00   999999
        VACANCIES 152320 149550 129500 163780 121950 132610 21261 104640 14474 14445 14733 24328 16014 13779 12595 15111 18578 999999 999999 999999 999999 999999 999999 999999 999999 999999 999999 0
        """,
    )
    return
end

function read_data(filename::String)
    data = DelimitedFiles.readdlm(filename)
    rows, columns = data[2:end, 1], data[1, 2:end]
    return Containers.DenseAxisArray(data[2:end, 2:end], rows, columns)
end

data = read_data(joinpath(@__DIR__, "transp.txt"))
#print(data)

function solve_transportation_problem(data::Containers.DenseAxisArray)
    # Get the set of supplies and demands
    O, D = axes(data)
    # Drop the EVACUEES and VACANCIES nodes from our sets
    O, D = setdiff(O, ["VACANCIES"]), setdiff(D, ["EVACUEES"])
    model = Model(HiGHS.Optimizer)
    set_silent(model)
    @variable(model, x[o in O, d in D] >= 0)
    # Remove arcs with "." cost by fixing them to 0.0.
    for o in O, d in D
        if data[o, d] == "."
            fix(x[o, d], 0.0; force = true)
        end
    end
    @objective(
        model,
        Min,
        sum(data[o, d] * x[o, d] for o in O, d in D if data[o, d] != "."),
    )
    @constraint(model, [o in O], sum(x[o, :]) == data[o, "EVACUEES"])
    @constraint(model, [d in D], sum(x[:, d]) <= data["VACANCIES", d])
    optimize!(model)
    
    
    # Construct the solution matrix
    solution_matrix = zeros(length(O), length(D))  # Initialize with zeros

    for (i, o) in enumerate(O)
        for (j, d) in enumerate(D)
            solution_matrix[i, j] = value(x[o, d])
        end
    end
    
    # Pretty print the solution in the format of the input
    print("    ", join(lpad.(D, 7, ' ')))
    for o in O
        print("\n", o)
        for d in D
            if isapprox(value(x[o, d]), 0.0; atol = 1e-6)
                print("      .")
            else
                print(" ", lpad(value(x[o, d]), 6, ' '))
            end
        end
    end
    return solution_matrix
end

print("\nSolution:\n")
solution_matrix = solve_transportation_problem(data)
println(solution_matrix)


Solution:
     広瀬中 仙台青 蒲町小 長町中 七郷小 南光台 新田小 仙台高 錦ケ丘 上杉山 宮城野 仙台商 岩切小 榴岡小 東長町 富沢小 富沢中   TSSW   TSDA   TSMP   TSTP   TSMN   TSHW   TSMS   TSH1   TSH2   TSH3
TTWR      .      .      .  894.0      .      .      .      .      .      .      .      .      .      .      .      .      .      .      .      .      .      .      .      .      .      .      .
ZUHO      .      .      .   60.0      .      .      .      .      .      .      .      .      .      .      .      .      .      .      .      .      .      .      .      .      .      .      .
OSAK      .      .      .      .      .      .      . 100000.0      .      .      .      .      .      .      .      .      .      .      .      .      .      .      .      .      .      .      .
TDAI      .      .      . 5000.0      .      .      .      .      .      .      .      .      .      .      .      .      .      .      .      .      .      .      .      .      .      .      .
AERT      .      .      .      .      .      .      .      .      . 

In [78]:
#DONE Total Cost Calculation
solution_cost = data.data[1:end-1,1:end-1]
matching_indicies = findall(x-> x == ".", solution_cost)
solution_cost[matching_indicies] .= 0
#solution_matrix .* solution_cost
total_cost_matrix = solution_cost.*solution_matrix
running_total = sum(total_cost_matrix)
initial_running_total = running_total

507759.75999999995

In [79]:
#DONE WHAT IS IT?
#print(solution_matrix)
#end_data[2:end-1, end]
end_data = Array(data)


ratios = solution_matrix ./ end_data[1:end-1, end]
print(ratios)
sum(ratios, dims=2)

#define original_population variable
original_population = data.data[1:end-1,end]
length(original_population)

#define capacity variable
capacity = data.data[end, 1:end-1]


#change original population by 0-5% for each location
#for ith population center sends population*ratio to each shelter
#1st population center to 4th shelter, population * ratio[1,4]

[0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 -0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.8043328748280605 0.0 0.0 0.0 0.0 0.0 0.01342450534334991 0.0 0.0 0.0 0.18224261982858958 0.0 0.0 0.0 0.0 0.0 0.0

27-element Vector{Any}:
 152320
 149550
 129500
 163780
 121950
 132610
  21261
 104640
  14474
  14445
  14733
  24328
  16014
      ⋮
  15111
  18578
 999999
 999999
 999999
 999999
 999999
 999999
 999999
 999999
 999999
 999999

In [80]:
#DONE define perturbed population and new solution matrix
epsilon_range = 0.03
perturbation_range = 1.0 .+ epsilon_range * 2 * (0.5 .- rand(size(original_population)[1]))
println("perturbation_range:")
println(perturbation_range)
perturbed_population = original_population .* perturbation_range
new_solution_matrix_decimal = perturbed_population .* ratios
new_solution_matrix = round.(new_solution_matrix_decimal)
println("new_solution_matrix:")
println(new_solution_matrix)

perturbation_range:
[1.0015902719482945, 0.9780613878920577, 0.9760880396356436, 1.0151388806247787, 1.024945346459239, 0.9949450696211394, 1.001001377229188, 1.0253976941300842, 1.0203044619904988, 0.97463596950443, 0.9846464359638, 1.0153979651259455, 0.9897455766717779, 0.9883881514484765, 1.001852376768266, 1.0280665443150387, 1.00772489762969, 0.9769845677120413, 1.0228645334661117, 0.9864053772799176, 1.0025451426011605, 0.9711352127625941, 1.0020925994320011, 1.0125281880155892]
new_solution_matrix:
[0.0 0.0 0.0 895.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 59.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 97609.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 5076.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 599.0

In [81]:
#DONE MONTE CARLO!
open(joinpath(@__DIR__, "shelter_matrix.txt"), "w") do io
    print(
        io,
        """
          .	Shelter1	Shelter2	Shelter3	Shelter4	Shelter5	Shelter6	Shelter7	Shelter8	Shelter9	Shelter10	Shelter11	Shelter12	Shelter13	Shelter14	Shelter15	Shelter16	Shelter17	Shelter18	Shelter19	Shelter20	Shelter21	Shelter22	Shelter23	Shelter24	Shelter25	Shelter26	Shelter27
Shelter1	0	8.71	17.86	15.36	19.08	15.36	16.39	8.09	2.73	11.28	13.88	18.83	13.93	13.29	16.05	15.12	14.11	10.77	13.78	13.57	13.84	13.62	16.1	14.51	16.04	16.59	16.8
Shelter2	8.71	0	12.25	9.75	13.47	6.95	10.65	0.97	10.83	5.54	8.22	11.3	6.83	7.63	11.66	12.78	11.73	5.16	5.37	11.31	8.18	6.51	10.49	8.85	10.43	10.98	11.19
Shelter3	17.86	12.25	0	6.08	1.21	8.98	4.52	11.28	19.99	7.13	4.43	7.67	13.16	5.04	5.11	8.12	7.55	7.12	9.42	7.76	4.4	12.44	3.64	3.85	4.08	3.6	3.87
Shelter4	15.36	9.75	6.08	0	7.28	9.86	8.15	8.78	15.59	5.39	4.96	11.27	12.48	4.34	2.74	3.52	2.47	4.62	8.71	2.42	4.63	11.76	3.09	4.69	2.5	3.59	3.26
Shelter5	19.08	13.47	1.21	7.28	0	9.76	5.31	12.49	21.2	8.34	5.65	7.86	13.61	6.25	6.3	9.31	8.74	8.34	10.63	8.96	5.62	13.31	4.84	5.06	5.27	4.79	5.07
Shelter6	15.36	6.95	8.98	9.86	9.76	0	4.65	7.48	17.48	5.16	5.98	4.73	4.9	5.69	11.03	12.9	11.85	7.16	2.49	11.8	6.33	4.2	10.11	6.14	9.78	10.1	10.38
Shelter7	16.39	10.65	4.52	8.15	5.31	4.65	0	9.68	18.52	5.45	3.36	3.51	9.27	4.16	8.81	11.16	10.1	6.68	6.02	10.09	3.73	8.69	7.34	3.61	7.78	7.33	7.61
Shelter8	8.09	0.97	11.28	8.78	12.49	7.48	9.68	0	10.22	4.56	7.25	11.84	8.04	6.66	10.69	11.81	10.76	4.18	5.91	10.34	7.21	7.72	9.52	7.88	9.45	10.01	10.21
Shelter9	2.73	10.83	19.99	15.59	21.2	17.48	18.52	10.22	0	13.4	16	20.96	16.06	15.41	18	15.47	15.47	12.89	15.91	15.74	15.97	15.74	18.23	16.63	18.16	18.72	18.92
Shelter10	11.28	5.54	7.13	5.39	8.34	5.16	5.45	4.56	13.4	0	2.8	7.91	7.11	2.23	7	8.42	7.37	2	3.36	7.32	2.78	6.4	5.76	3.43	5.78	6.25	6.54
Shelter11	13.88	8.22	4.43	4.96	5.65	5.98	3.36	7.25	16	2.8	0	6.48	9.87	0.91	6.25	7.96	6.91	3.43	5.38	6.89	0.39	9.16	4.88	0.72	5	5.37	5.65
Shelter12	18.83	11.3	7.67	11.27	7.86	4.73	3.51	11.84	20.96	7.91	6.48	0	7.54	7.28	12	14.28	13.22	9.69	6.86	13.21	6.85	7.29	10.54	6.73	10.97	10.49	10.77
Shelter13	13.93	6.83	13.16	12.48	13.61	4.9	9.27	8.04	16.06	7.11	9.87	7.54	0	9.3	14.12	15.51	14.46	9.11	4.33	14.41	9.86	1.02	12.85	10.31	12.88	13.34	13.63
Shelter14	13.29	7.63	5.04	4.34	6.25	5.69	4.16	6.66	15.41	2.23	0.91	7.28	9.3	0	5.64	7.35	6.29	2.63	5.29	6.27	0.65	8.59	4.36	1.33	4.39	4.85	5.14
Shelter15	16.05	11.66	5.11	2.74	6.3	11.03	8.81	10.69	18	7	6.25	12	14.12	5.64	0	3.06	2.95	6.53	10.35	3.64	5.93	13.41	2.3	5.63	2.09	2.36	1.93
Shelter16	15.12	12.78	8.12	3.52	9.31	12.9	11.16	11.81	15.47	8.42	7.96	14.28	15.51	7.35	3.06	0	1.27	7.65	11.74	2.52	7.64	14.8	5.31	7.7	4.81	5.37	4.94
Shelter17	14.11	11.73	7.55	2.47	8.74	11.85	10.1	10.76	15.47	7.37	6.91	13.22	14.46	6.29	2.95	1.27	0	6.6	10.69	1.44	6.58	13.75	4.74	6.65	4.08	5.16	4.56
Shelter18	10.77	5.16	7.12	4.62	8.34	7.16	6.68	4.18	12.89	2	3.43	9.69	9.11	2.63	6.53	7.65	6.6	0	5.36	6.18	3.19	8.4	5.36	3.85	5.3	5.85	6.06
Shelter19	13.78	5.37	9.42	8.71	10.63	2.49	6.02	5.91	15.91	3.36	5.38	6.86	4.33	5.29	10.35	11.74	10.69	5.36	0	10.64	5.45	3.61	9.07	6.01	9.1	9.57	9.86
Shelter20	13.57	11.31	7.76	2.42	8.96	11.8	10.09	10.34	15.74	7.32	6.89	13.21	14.41	6.27	3.64	2.52	1.44	6.18	10.64	0	6.57	13.7	4.96	6.63	4.22	5.37	4.78
Shelter21	13.84	8.18	4.4	4.63	5.62	6.33	3.73	7.21	15.97	2.78	0.39	6.85	9.86	0.65	5.93	7.64	6.58	3.19	5.45	6.57	0	9.14	4.65	0.7	4.68	5.14	5.44
Shelter22	13.62	6.51	12.44	11.76	13.31	4.2	8.69	7.72	15.74	6.4	9.16	7.29	1.02	8.59	13.41	14.8	13.75	8.4	3.61	13.7	9.14	0	12.13	9.59	12.16	12.62	12.92
Shelter23	16.1	10.49	3.64	3.09	4.84	10.11	7.34	9.52	18.23	5.76	4.88	10.54	12.85	4.36	2.3	5.31	4.74	5.36	9.07	4.96	4.65	12.13	0	4.32	1.27	0.55	1.41
Shelter24	14.51	8.85	3.85	4.69	5.06	6.14	3.61	7.88	16.63	3.43	0.72	6.73	10.31	1.33	5.63	7.7	6.65	3.85	6.01	6.63	0.7	9.59	4.32	0	4.38	4.81	5.06
Shelter25	16.04	10.43	4.08	2.5	5.27	9.78	7.78	9.45	18.16	5.78	5	10.97	12.88	4.39	2.09	4.81	4.08	5.3	9.1	4.22	4.68	12.16	1.27	4.38	0	1.69	0.76
Shelter26	16.59	10.98	3.6	3.59	4.79	10.1	7.33	10.01	18.72	6.25	5.37	10.49	13.34	4.85	2.36	5.37	5.16	5.85	9.57	5.37	5.14	12.62	0.55	4.81	1.69	0	1.08
Shelter27	16.8	11.19	3.87	3.26	5.07	10.38	7.61	10.21	18.92	6.54	5.65	10.77	13.63	5.14	1.93	4.94	4.56	6.06	9.86	4.78	5.44	12.92	1.41	5.06	0.76	1.08	0
        """,
    )
    return
end

function read_data(filename::String)
    data = DelimitedFiles.readdlm(filename)
    rows, columns = data[2:end, 1], data[1, 2:end]
    return Containers.DenseAxisArray(data[2:end, 2:end], rows, columns)
end

shelter_matrix = read_data(joinpath(@__DIR__, "shelter_matrix.txt"))
#println(shelter_matrix)

assigned_shelters_pop = sum(new_solution_matrix, dims=1)

negativity_check = (transpose(capacity) - assigned_shelters_pop)[1:16]

println(negativity_check .< 0)
overflow_shelters = findall(<(0), negativity_check) #indices of shelters that are over capacity

function minfinder(x)
    if x > 0
        return x
    else 
        return 999999 #return huge number so 0 isn't chosen
    end
end
println(overflow_shelters)
for i in overflow_shelters
    #println("Row ",i,", Overflow Value: ",-testing123[i]) THIS WAS WHICH ONE IS OVERFLOWING!!!
    row = shelter_matrix.data[i,:]
    (minvalue,minindex) = findmin(minfinder,row)
    println(minvalue)
    println(minindex)
    println(row)
    
    overflow_population = assigned_shelters_pop[i] - capacity[i]
    assigned_shelters_pop[minindex] += overflow_population
    assigned_shelters_pop[i] -= overflow_population
    running_total += overflow_population * minvalue
     
    
    is_pop_moving = assigned_shelters_pop[minindex] > capacity[minindex]
    println(is_pop_moving)
    looping_number = 0
    
    while is_pop_moving && looping_number <=20
        println(looping_number)
        oldindex = minindex
        row = shelter_matrix.data[oldindex,:]
        (minvalue,minindex) = findmin(minfinder,row)
        overflow_population = assigned_shelters_pop[oldindex] - capacity[oldindex]
        assigned_shelters_pop[minindex] += overflow_population
        assigned_shelters_pop[oldindex] -= overflow_population
        println(assigned_shelters_pop)
        is_pop_moving = assigned_shelters_pop[minindex] > capacity[minindex]
        println(is_pop_moving)
        running_total += overflow_population * minvalue
        looping_number += 1
    end
    if is_pop_moving
        final_overflow_count = assigned_shelters_pop[minindex] - capacity[minindex]
        print("Final Overflow Population Number:") 
        println(final_overflow_count)
    end
end
println("Total Cost:")
println(running_total)
println("Cost Difference between Final Cost and initial cost: ")
println(running_total - initial_running_total)
println("Percentage")
print(initial_running_total / running_total)


Bool[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0]
[11, 14]
0.39
21
Any[13.88, 8.22, 4.43, 4.96, 5.65, 5.98, 3.36, 7.25, 16, 2.8, 0, 6.48, 9.87, 0.91, 6.25, 7.96, 6.91, 3.43, 5.38, 6.89, 0.39, 9.16, 4.88, 0.72, 5, 5.37, 5.65]
false
0.65
21
Any[13.29, 7.63, 5.04, 4.34, 6.25, 5.69, 4.16, 6.66, 15.41, 2.23, 0.91, 7.28, 9.3, 0, 5.64, 7.35, 6.29, 2.63, 5.29, 6.27, 0.65, 8.59, 4.36, 1.33, 4.39, 4.85, 5.14]
false
Total Cost:
507956.18999999994
Cost Difference between Final Cost and initial cost: 
196.42999999999302
Percentage
0.9996132934220173

In [85]:
#randomly change population for variance. Not monte carlo but good simulation. 

import JuMP
using JuMP
import Random
using Random
import DelimitedFiles
using DelimitedFiles
import Statistics
using Statistics
import HiGHS

open(joinpath(@__DIR__, "transp.txt"), "w") do io
    print(
        io,
        """
             . 広瀬中	仙台青	蒲町小	長町中	七郷小	南光台	新田小	仙台高	錦ケ丘	上杉山	宮城野	仙台商	岩切小	榴岡小	東長町	富沢小	富沢中	TSSW	TSDA	TSMP	TSTP	TSMN	TSHW	TSMS	TSH1	TSH2	TSH3  EVACUEES
          TTWR .	.	.	2.85	.	.	.	4.80	.	1.65	2.31	.	.	1.47	4.33	.	4.38	1.39	4.37	3.94	1.99	.	3.88	2.46	3.76	4.34	4.49	894
          ZUHO .	.	.	2.30	.	.	.	4.58	.	2.36	3.37	.	.	2.54	4.12	4.54	3.57	1.20	.	3.00	3.03	.	4.03	3.44	3.85	4.53	4.64	60
          OSAK .	2.05	.	.	.	.	.	1.46	.	2.65	.	.	.	4.45	.	.	.	2.06	4.17	.	.	.	.	.	.	.	.	100000
          TDAI .	.	.	2.54	.	.	.	4.82	.	1.93	2.61	.	.	1.78	4.11	4.98	4.03	1.35	4.69	3.57	2.27	.	3.77	2.70	3.63	4.25	4.39	5000
          AERT .	.	.	3.56	.	4.54	4.64	4.84	.	1.20	1.77	.	.	0.93	4.86	.	.	1.75	3.70	4.72	1.50	.	4.22	2.06	4.13	4.64	4.81	584
          TEPC .	4.81	.	3.86	.	4.33	4.78	4.47	.	0.78	2.04	.	.	1.25	.	.	.	1.58	3.36	4.92	1.82	.	4.64	2.40	4.54	.	.	12500
          SIPO .	.	.	.	.	.	.	.	.	.	.	3.68	.	.	.	.	.	.	.	.	.	4.29	.	.	.	.	.	1653
          MEDI .	3.88	.	3.89	.	4.99	.	3.45	.	0.99	3.21	.	.	2.39	.	.	.	0.57	3.65	4.54	2.97	.	.	3.53	.	.	.	346
          SSTA .	.	4.83	3.35	.	4.71	4.60	.	.	1.47	1.66	.	.	0.81	4.61	.	4.96	1.87	3.94	4.58	1.36	.	3.95	1.90	3.86	4.37	4.54	75608
          AQUA .	.	.	.	.	.	4.97	.	.	.	.	.	4.19	.	.	.	.	.	.	.	.	.	.	.	.	.	.	913
          MOPS .	.	.	.	.	.	.	.	.	.	.	.	5.09	.	.	.	.	.	.	.	.	.	.	.	.	.	.	10000
          YURT .	.	.	.	.	2.70	.	.	.	.	.	1.03	.	.	.	.	.	.	2.58	.	.	0.49	.	.	.	.	.	11467
          CAST .	4.51	.	2.90	.	.	.	3.87	.	2.68	4.27	.	.	3.42	4.85	4.72	3.77	1.15	.	3.01	3.95	.	4.95	4.41	4.76	.	.	87
          RAKU .	.	2.96	3.98	4.13	4.78	3.19	.	.	3.13	0.68	.	.	1.21	4.39	.	.	3.79	4.74	.	0.67	.	3.24	0.13	3.27	3.48	3.68	27000
          TLSW 9.36	4.01	6.62	3.51	7.86	5.56	6.33	3.49	9.50	1.55	3.51	7.02	8.45	2.66	5.30	5.65	4.69	0.00	4.21	4.02	3.24	6.61	5.10	3.77	4.95	5.59	5.72   999999
          TLDA .	4.38	7.45	7.22	8.38	1.86	4.90	4.57	.	2.80	4.08	3.21	5.62	3.91	8.53	9.66	8.71	4.23	0.00	8.16	4.17	2.66	7.74	4.63	7.69	8.09	8.28   999999
          TLMP .	7.36	7.04	1.70	8.21	9.21	8.68	6.67	9.68	5.35	5.76	.	.	5.11	3.11	1.75	0.89	4.02	8.13	0.00	5.41	.	4.03	5.60	3.80	4.45	4.45   999999
          TLTP .	6.56	3.63	3.96	4.79	4.38	3.37	6.26	.	2.47	0.36	7.35	5.98	0.58	4.66	6.50	5.66	3.23	4.16	5.47	0.00	6.78	3.64	0.58	3.64	3.95	4.14   999999
          TLMN .	5.20	9.87	9.82	.	3.07	6.79	5.70	.	5.35	6.67	0.62	6.40	6.57	.	.	.	6.62	2.66	.	6.80	0.00	.	7.22	.	.	.   999999
          TLHW .	9.02	3.23	2.47	4.29	8.01	5.98	8.50	.	5.38	3.86	.	8.86	3.81	1.38	4.04	3.62	5.05	7.72	4.03	3.64	.	0.00	3.37	0.23	0.50	0.62   999999
          TLMS .	7.13	3.05	4.07	4.20	4.65	3.12	6.83	.	3.05	0.55	7.77	5.86	1.14	4.51	6.56	5.77	3.76	4.61	5.66	0.58	7.19	3.37	0.00	3.40	3.62	3.81   999999
          TLH1 .	8.88	3.43	2.24	4.50	8.02	6.09	8.35	.	5.29	3.87	.	8.96	3.78	1.23	3.84	3.40	4.90	7.68	3.80	3.64	.	0.23	3.40	0.00	0.71	0.79   999999
          TLH2 .	9.49	3.02	2.94	3.99	8.27	6.04	8.98	.	5.80	4.13	.	8.92	4.17	1.59	4.32	3.98	5.53	8.07	4.45	3.95	.	0.50	3.62	0.71	0.00	0.20   999999
          TLH3 .	9.63	3.16	2.97	4.09	8.46	6.23	9.11	.	5.97	4.33	.	9.10	4.35	1.50	4.23	3.94	5.66	8.25	4.44	4.14	.	0.62	3.81	0.79	0.20	0.00   999999
        VACANCIES 152320 149550 129500 163780 121950 132610 21261 104640 14474 14445 14733 24328 16014 13779 12595 15111 18578 999999 999999 999999 999999 999999 999999 999999 999999 999999 999999 0
        """,
    )
    return
end

function read_data(filename::String)
    data = DelimitedFiles.readdlm(filename)
    rows, columns = data[2:end, 1], data[1, 2:end]
    return Containers.DenseAxisArray(data[2:end, 2:end], rows, columns)
end

data = read_data(joinpath(@__DIR__, "transp.txt"))
print(data)

function solve_transportation_problem(data::Containers.DenseAxisArray)
    # Get the set of supplies and demands
    O, D = axes(data)
    # Drop the EVACUEES and VACANCIES nodes from our sets
    O, D = setdiff(O, ["VACANCIES"]), setdiff(D, ["EVACUEES"])
    model = Model(HiGHS.Optimizer)
    set_silent(model)
    @variable(model, x[o in O, d in D] >= 0)
    # Remove arcs with "." cost by fixing them to 0.0.
    for o in O, d in D
        if data[o, d] == "."
            fix(x[o, d], 0.0; force = true)
        end
    end
    @objective(
        model,
        Min,
        sum(data[o, d] * x[o, d] for o in O, d in D if data[o, d] != "."),
    )
    @constraint(model, [o in O], sum(x[o, :]) == data[o, "EVACUEES"])
    @constraint(model, [d in D], sum(x[:, d]) <= data["VACANCIES", d])
    optimize!(model)
    # Pretty print the solution in the format of the input
    print("    ", join(lpad.(D, 7, ' ')))
    for o in O
        print("\n", o)
        for d in D
            if isapprox(value(x[o, d]), 0.0; atol = 1e-6)
                print("      .")
            else
                print(" ", lpad(value(x[o, d]), 6, ' '))
            end
        end
    end
    return
end

# Print the original cost
original_cost = calculate_total_distance(data, data)
println("\nOriginal Total Cost: $original_cost")

#-------------
# Specify the number of Monte Carlo trials and epsilon range
num_trials = 50  # Adjust this as needed
epsilon_range = 0.07  # range for population perturbations

# Define a function to perturb the population data with randomization
function perturb_population(data::Containers.DenseAxisArray, perturbations::Dict{String, Float64})
    perturbed_data = deepcopy(data)
print("printing perturbations")
    for (o, perturbation) in perturbations
        # Apply the perturbation to the population
        original_population = data[o, "EVACUEES"]
        perturbed_population = original_population * (1.0 + perturbation)
        println(perturbation)

        # Update the perturbed population in the data
        perturbed_data[o, "EVACUEES"] = perturbed_population
    end

    return perturbed_data
end

# Define a function to run the Monte Carlo simulation with population perturbations
function run_monte_carlo_simulation(data::Containers.DenseAxisArray, num_trials::Int, epsilon_range::Float64)
    # Create an array to store the results from each trial
    total_distances = Float64[]

    # Loop through the specified number of trials
    for trial in 1:num_trials
        println("\nTrial $trial:")

        # Set a new random seed for each trial to ensure different perturbations
        Random.seed!(trial)

        # Create a random perturbation for each DL
        perturbations = Dict{String, Float64}()
        total_perturbation = 0.0  # Initialize total perturbation for this trial
        for o in axes(data)[1]
            if o != "EVACUEES" && o != "VACANCIES"  # Exclude "VACANCIES" from perturbation
                # Generate a random perturbation within the epsilon range
                perturbation = (2.0 * epsilon_range * rand() - epsilon_range)  # Random float within the range
                perturbations[o] = perturbation
                println("Perturbation for $o: $perturbation")

                # Calculate perturbation contribution to total perturbation
                original_population = data[o, "EVACUEES"]
                perturbation_contribution = original_population * perturbation
                total_perturbation += perturbation_contribution
            end
        end

        # Print the total perturbation for this trial
        println("Total Perturbation for Trial $trial: $total_perturbation")

        # Perturb the population data using the perturbations
        perturbed_data = perturb_population(data, perturbations)

        # Solve the transportation problem for the perturbed data
        solve_transportation_problem(perturbed_data)

        # Calculate and record the total distance for this trial
        total_distance = calculate_total_distance(data, perturbed_data)
        push!(total_distances, total_distance)
    end
    return total_distances
end

# Run the Monte Carlo simulation
results = run_monte_carlo_simulation(data, num_trials, epsilon_range)

# Output the results
println("\nSimulation Results:")
println("Total Distances for Each Trial: ", results)

average_total_cost = mean(results)
println("Average Total Cost for All Trials: $average_total_cost")


2-dimensional DenseAxisArray{Any,2,...} with index sets:
    Dimension 1, Any["TTWR", "ZUHO", "OSAK", "TDAI", "AERT", "TEPC", "SIPO", "MEDI", "SSTA", "AQUA"  …  "TLDA", "TLMP", "TLTP", "TLMN", "TLHW", "TLMS", "TLH1", "TLH2", "TLH3", "VACANCIES"]
    Dimension 2, Any["広瀬中", "仙台青", "蒲町小", "長町中", "七郷小", "南光台", "新田小", "仙台高", "錦ケ丘", "上杉山"  …  "TSDA", "TSMP", "TSTP", "TSMN", "TSHW", "TSMS", "TSH1", "TSH2", "TSH3", "EVACUEES"]
And data, a 25×28 Matrix{Any}:
       "."        "."        "."       2.85        "."        "."       "."       4.8        "."      1.65      2.31       "."       "."      1.47      4.33       "."      4.38       1.39       4.37       3.94       1.99        "."       3.88       2.46       3.76       4.34       4.49     894
       "."        "."        "."       2.3         "."        "."       "."       4.58       "."      2.36      3.37       "."       "."      2.54      4.12      4.54      3.57       1.2         "."       3.0        3.03        "."       4.03       3

LoadError: UndefVarError: calculate_total_distance not defined

In [84]:
#always run cell above first
import Random
using Random
max_deviation = 0.05
Random.seed!(123)
rand()

O, D = axes(data)
# Drop the EVACUEES and VACANCIES nodes from our sets
O, D = setdiff(O, ["VACANCIES"]), setdiff(D, ["EVACUEES"])
# Remove arcs with "." cost by fixing them to 0.0.
for o in O, d in D
    if data[o, d] == "."
        data[o,d] = 0.0
    end
    new_pop = 1-rand()*max_deviation
    print(new_pop)
    data[o,d] *= new_pop
end
print(rand())
print(data)
    
#Shrink (*95-100%), OR, make sure shelter cap >> DL pop
#wrap the entire cell in a for loop. Keep track of all costs. average out the cost. 

0.97065966212733260.9554560650953610.99045466504871180.97371688042289760.98047058622843270.99775909974912550.95333233563614170.97097200090627290.98363810603718580.97365020406015070.95818857124739240.99795469319861540.97673992473093880.98186753367907790.9948897696755620.96399487202548350.9713190378765680.96677657606365350.985231674762260.98617012769125170.9508282144440080.95595512545920970.98829915971129720.98095253103569460.9934028133730250.99558544904338610.98432475427461350.97681951278375420.96431820387568960.98970375452566450.9954724417891110.97090438288061780.98442762496474730.99394262397409360.9897735091339820.98066549185455230.99907140002050310.99639096381492960.95428767070281030.98063129688732180.96140418220776110.98113083702836230.95622225002201910.98677159298190340.95490613377347920.95676534539844150.96338809243910150.99484015767495850.97062075318955740.95952836220292690.98880795986397360.9570737769507660.98122915381477430.96616464704802270.95395577879366440.96467694292159540.

0.9608077415820610.96915134403676870.97254922142306430.96095355090467370.9781978252677380.99499535341738870.99350191683422930.99329511331079780.9985269892167250.98552034388366650.98667284518223930.99061381213589870.9683486242512580.95758265298192070.96788090163961480.9506152747727570.99552876719164160.95766449011250930.95733813776397790.9516805675900780.99393766130618380.96789025030619840.98986278391549460.95374788749755680.95050747692205560.9845349451778150.95394274675347420.96830219273232120.98780986808120260.9716713187787040.95471367809694640.98229739761684230.97914631108536070.96000801913395670.96875429339304460.96683739006483350.98413570321158380.99823001583001480.99108368699480150.9528037168677470.98464833530753550.95265434480959820.98274578817101490.98056310563148890.9600247460477970.95788400072404560.97723000806789940.96622520213577370.97868340391865580.96159587292469060.99442235186634680.96187266166298080.98339847650024360.99069575224952190.96928736490166990.95333940557414950.

      0.0                     4.3109934484134795       0.0                      2.8582219085567777       0.0                      0.0                     0.0                      3.8024708367664393      0.0                    2.6547766828614727      4.157040117840961        0.0                     0.0                    3.3035111349688586      4.67242801409108        4.551204797287443       3.7329043999817624       1.1264573478963644       0.0                      2.9460554863561677       3.7844104864510175       0.0                      4.753091080710651         4.19810311591398          4.752744630106487         0.0                       0.0                     87
      0.0                     0.0                      2.892537247518189        3.9364563649159656       4.113741243761644        4.741025002753437       3.1641620689280643       0.0                     0.0                    3.0389691532785106      0.6512067755101444       0.0                     0.0                    1.1

In [82]:
#Monte Carlo 

#I get some suggestions for route based off of my LP (tells me cost)
#What if durations changed randomly (wiggle by 5%)
#big for loop for several different levels 5, 10, 100
#Stick with the same routes same amounts of people in each place; what will it look like in this iteration
#repeat 10,000 times
#total duration averaged out amongst 10,000 trials
#variation in duration over different levels 
#standard deviation
#also wiggle the capacities
#-----------------------------------------
#remove routes randomly
#each iteration run the LP over and over again
#randomly at each for loop remove random route and find duration
#tells me how my LP does

#Scenario 1: "call shelter and ask if their full"
#setting standard
#fixed number of people
#get each person to walk to the nearest shelter
#second person if shelter is full go to the next shelter
#repeat over and over again for population P

#Scenario 2:"go to the shelter and realize it is full"
#get there and realize its full and then go to next shelter

In [83]:
#DONE VISUALIZATION
using Conda
Conda.runconda(`install folium -c conda-forge`)

import PyCall
using PyCall
fol = pyimport("folium")

min_lon, max_lon = 140.4, 141.14
min_lat, max_lat = 38.1, 38.5

# Create a folium Map object
m = fol.Map(
    max_bounds=true,
    location=[38.150223, 140.869415],
    zoom_start=10,
    min_lat=min_lat,
    max_lat=max_lat,
    min_lon=min_lon,
    max_lon=max_lon,
)

# Add CircleMarkers to the map
fol.CircleMarker([max_lat, min_lon], tooltip="Upper Left Corner").add_to(m)
fol.CircleMarker([min_lat, min_lon], tooltip="Lower Left Corner").add_to(m)
fol.CircleMarker([min_lat, max_lon], tooltip="Lower Right Corner").add_to(m)
fol.CircleMarker([max_lat, max_lon], tooltip="Upper Right Corner").add_to(m)

#Icons
fg = fol.FeatureGroup(name="Icon collection").add_to(m)
fol.Marker(location=(38.25523, 140.869415)).add_to(fg)

# Load and display the GeoJSON file
sendai_border = "/Users/jakechon/Downloads/04100_sendai-shi_2022_other_1_op/sendai_border.geojson"
fol.GeoJson(sendai_border).add_to(m)

fol.LayerControl().add_to(m)

#Population Distribution use https://www.researchgate.net/publication/329350011_Dynamic_Integrated_Model_for_Disaster_Management_and_Socioeconomic_Analysis_DIM2SEA

#UX
fol.GeoJson(sendai_border, zoom_on_click=true).add_to(m)#make it so that it isn't only the lines

# Display the map
m


In [ ]:
#With Transshipment Nodes
using JuMP
import DelimitedFiles
import HiGHS

open(joinpath(@__DIR__, "transp.txt"), "w") do io
    print(
        io,
        """
             . 広瀬中	仙台青	蒲町小	長町中	七郷小	南光台	新田小	仙台高	錦ケ丘	上杉山	宮城野	仙台商	岩切小	榴岡小	東長町	富沢小	富沢中	TSSW	TSDA	TSMP	TSTP	TSMN	TSHW	TSMS	TSH1	TSH2	TSH3  EVACUEES
          TTWR .	.	.	2.85	.	.	.	4.80	.	1.65	2.31	.	.	1.47	4.33	.	4.38	1.39	4.37	3.94	1.99	.	3.88	2.46	3.76	4.34	4.49	894
          ZUHO .	.	.	2.30	.	.	.	4.58	.	2.36	3.37	.	.	2.54	4.12	4.54	3.57	1.20	.	3.00	3.03	.	4.03	3.44	3.85	4.53	4.64	60
          OSAK .	2.05	.	.	.	.	.	1.46	.	2.65	.	.	.	4.45	.	.	.	2.06	4.17	.	.	.	.	.	.	.	.	100000
          TDAI .	.	.	2.54	.	.	.	4.82	.	1.93	2.61	.	.	1.78	4.11	4.98	4.03	1.35	4.69	3.57	2.27	.	3.77	2.70	3.63	4.25	4.39	5000
          AERT .	.	.	3.56	.	4.54	4.64	4.84	.	1.20	1.77	.	.	0.93	4.86	.	.	1.75	3.70	4.72	1.50	.	4.22	2.06	4.13	4.64	4.81	584
          TEPC .	4.81	.	3.86	.	4.33	4.78	4.47	.	0.78	2.04	.	.	1.25	.	.	.	1.58	3.36	4.92	1.82	.	4.64	2.40	4.54	.	.	12500
          SIPO .	.	.	.	.	.	.	.	.	.	.	3.68	.	.	.	.	.	.	.	.	.	4.29	.	.	.	.	.	1653
          MEDI .	3.88	.	3.89	.	4.99	.	3.45	.	0.99	3.21	.	.	2.39	.	.	.	0.57	3.65	4.54	2.97	.	.	3.53	.	.	.	346
          SSTA .	.	4.83	3.35	.	4.71	4.60	.	.	1.47	1.66	.	.	0.81	4.61	.	4.96	1.87	3.94	4.58	1.36	.	3.95	1.90	3.86	4.37	4.54	75608
          AQUA .	.	.	.	.	.	4.97	.	.	.	.	.	4.19	.	.	.	.	.	.	.	.	.	.	.	.	.	.	913
          MOPS .	.	.	.	.	.	.	.	.	.	.	.	5.09	.	.	.	.	.	.	.	.	.	.	.	.	.	.	10000
          YURT .	.	.	.	.	2.70	.	.	.	.	.	1.03	.	.	.	.	.	.	2.58	.	.	0.49	.	.	.	.	.	11467
          CAST .	4.51	.	2.90	.	.	.	3.87	.	2.68	4.27	.	.	3.42	4.85	4.72	3.77	1.15	.	3.01	3.95	.	4.95	4.41	4.76	.	.	87
          RAKU .	.	2.96	3.98	4.13	4.78	3.19	.	.	3.13	0.68	.	.	1.21	4.39	.	.	3.79	4.74	.	0.67	.	3.24	0.13	3.27	3.48	3.68	27000
          TLSW 9.36	4.01	6.62	3.51	7.86	5.56	6.33	3.49	9.50	1.55	3.51	7.02	8.45	2.66	5.30	5.65	4.69	0.00	4.21	4.02	3.24	6.61	5.10	3.77	4.95	5.59	5.72   999999
          TLDA .	4.38	7.45	7.22	8.38	1.86	4.90	4.57	.	2.80	4.08	3.21	5.62	3.91	8.53	9.66	8.71	4.23	0.00	8.16	4.17	2.66	7.74	4.63	7.69	8.09	8.28   999999
          TLMP .	7.36	7.04	1.70	8.21	9.21	8.68	6.67	9.68	5.35	5.76	.	.	5.11	3.11	1.75	0.89	4.02	8.13	0.00	5.41	.	4.03	5.60	3.80	4.45	4.45   999999
          TLTP .	6.56	3.63	3.96	4.79	4.38	3.37	6.26	.	2.47	0.36	7.35	5.98	0.58	4.66	6.50	5.66	3.23	4.16	5.47	0.00	6.78	3.64	0.58	3.64	3.95	4.14   999999
          TLMN .	5.20	9.87	9.82	.	3.07	6.79	5.70	.	5.35	6.67	0.62	6.40	6.57	.	.	.	6.62	2.66	.	6.80	0.00	.	7.22	.	.	.   999999
          TLHW .	9.02	3.23	2.47	4.29	8.01	5.98	8.50	.	5.38	3.86	.	8.86	3.81	1.38	4.04	3.62	5.05	7.72	4.03	3.64	.	0.00	3.37	0.23	0.50	0.62   999999
          TLMS .	7.13	3.05	4.07	4.20	4.65	3.12	6.83	.	3.05	0.55	7.77	5.86	1.14	4.51	6.56	5.77	3.76	4.61	5.66	0.58	7.19	3.37	0.00	3.40	3.62	3.81   999999
          TLH1 .	8.88	3.43	2.24	4.50	8.02	6.09	8.35	.	5.29	3.87	.	8.96	3.78	1.23	3.84	3.40	4.90	7.68	3.80	3.64	.	0.23	3.40	0.00	0.71	0.79   999999
          TLH2 .	9.49	3.02	2.94	3.99	8.27	6.04	8.98	.	5.80	4.13	.	8.92	4.17	1.59	4.32	3.98	5.53	8.07	4.45	3.95	.	0.50	3.62	0.71	0.00	0.20   999999
          TLH3 .	9.63	3.16	2.97	4.09	8.46	6.23	9.11	.	5.97	4.33	.	9.10	4.35	1.50	4.23	3.94	5.66	8.25	4.44	4.14	.	0.62	3.81	0.79	0.20	0.00   999999
        VACANCIES 15232 14955 12950 16378 12195 13261 21261 10464 14474 14445 14733 24328 16014 13779 12595 15111 18578 999999 999999 999999 999999 999999 999999 999999 999999 999999 999999 0
        """,
    )
    return
end

function read_data(filename::String)
    data = DelimitedFiles.readdlm(filename)
    rows, columns = data[2:end, 1], data[1, 2:end]
    return Containers.DenseAxisArray(data[2:end, 2:end], rows, columns)
end

data = read_data(joinpath(@__DIR__, "transp.txt"))
print(data)

function solve_transportation_problem(data::Containers.DenseAxisArray)
    # Get the set of supplies and demands
    O, D = axes(data)
    # Drop the EVACUEES and VACANCIES nodes from our sets
    O, D = setdiff(O, ["VACANCIES"]), setdiff(D, ["EVACUEES"])
    model = Model(HiGHS.Optimizer)
    set_silent(model)
    @variable(model, x[o in O, d in D] >= 0)
    # Remove arcs with "." cost by fixing them to 0.0.
    for o in O, d in D
        if data[o, d] == "."
            fix(x[o, d], 0.0; force = true)
        end
    end
    @objective(
        model,
        Min,
        sum(data[o, d] * x[o, d] for o in O, d in D if data[o, d] != "."),
    )
    @constraint(model, [o in O], sum(x[o, :]) == data[o, "EVACUEES"])
    @constraint(model, [d in D], sum(x[:, d]) <= data["VACANCIES", d])
    optimize!(model)
    # Pretty print the solution in the format of the input
    print("    ", join(lpad.(D, 7, ' ')))
    for o in O
        print("\n", o)
        for d in D
            if isapprox(value(x[o, d]), 0.0; atol = 1e-6)
                print("      .")
            else
                print(" ", lpad(value(x[o, d]), 6, ' '))
            end
        end
    end
    return
end

print("\nSolution:\n")
solve_transportation_problem(data)

#write for loop

#Derivative Calculation (Pathway Cost)

function calculate_numerical_derivative(data::JuMP.Containers.DenseAxisArray{Any, 2}, epsilon::Float64 = 1)
    O, D = axes(data)
    O, D = setdiff(O, ["VACANCIES"]), setdiff(D, ["EVACUEES"])
    num_rows, num_cols = length(O), length(D)
    derivative_matrix = zeros(Float64, num_rows, num_cols)

    for (i, o) in enumerate(O), (j, d) in enumerate(D)
        original_cost = data[o, d]

        if original_cost isa Number
            # Increase the cost slightly and re-solve
            data[o, d] = original_cost + epsilon
            increased_cost_model = Model(HiGHS.Optimizer)
            set_silent(increased_cost_model)
            @variable(increased_cost_model, x[ii in O, jj in D] >= 0)

            for ii in O, jj in D
                if data[ii, jj] == "."
                    fix(x[ii, jj], 0.0; force = true)
                end
            end

            @objective(
                increased_cost_model,
                Min,
                sum(data[ii, jj] * x[ii, jj] for ii in O, jj in D if data[ii, jj] isa Number),
            )

            @constraint(increased_cost_model, [ii in O], sum(x[ii, :]) == data[ii, "EVACUEES"])
            @constraint(increased_cost_model, [jj in D], sum(x[:, jj]) <= data["VACANCIES", jj])

            optimize!(increased_cost_model)
            increased_cost_objective = objective_value(increased_cost_model)

            # Decrease the cost slightly and re-solve
            data[o, d] = original_cost - epsilon
            decreased_cost_model = Model(HiGHS.Optimizer)
            set_silent(decreased_cost_model)
            @variable(decreased_cost_model, x[ii in O, jj in D] >= 0)

            for ii in O, jj in D
                if data[ii, jj] == "."
                    fix(x[ii, jj], 0.0; force = true)
                end
            end

            @objective(
                decreased_cost_model,
                Min,
                sum(data[ii, jj] * x[ii, jj] for ii in O, jj in D if data[ii, jj] isa Number),
            )

            @constraint(decreased_cost_model, [ii in O], sum(x[ii, :]) == data[ii, "EVACUEES"])
            @constraint(decreased_cost_model, [jj in D], sum(x[:, jj]) <= data["VACANCIES", jj])

            optimize!(decreased_cost_model)
            decreased_cost_objective = objective_value(decreased_cost_model)

            # Calculate the numerical derivative
            derivative = (increased_cost_objective - decreased_cost_objective) / (2 * epsilon)
            derivative_matrix[i, j] = derivative

            # Reset the original cost
            data[o, d] = original_cost
        end
    end

    return derivative_matrix
end

# Calculate and print the numerical derivative matrix
epsilon = 1.0
derivative_matrix = calculate_numerical_derivative(data, epsilon)
println("\nNumerical Derivative Matrix for Pathway:")
for row in eachrow(derivative_matrix)
    println(row)
end

# Derivative Calculation (Shelter Capacity)

function calculate_numerical_capacity_derivative(data::JuMP.Containers.DenseAxisArray{Any, 2}, epsilon::Float64 = 1000)
    O, D = axes(data)
    O, D = setdiff(O, ["VACANCIES"]), setdiff(D, ["EVACUEES"])
    num_rows, num_cols = length(O), length(D)
    derivative_matrix = zeros(Float64, num_rows, num_cols)

    for (i, o) in enumerate(O), (j, d) in enumerate(D)
        original_capacity = data["VACANCIES", d]

        if original_capacity isa Number
            # Increase the capacity slightly and re-solve
            data["VACANCIES", d] = original_capacity + epsilon
            increased_capacity_model = Model(HiGHS.Optimizer)
            set_silent(increased_capacity_model)
            @variable(increased_capacity_model, x[ii in O, jj in D] >= 0)

            for ii in O, jj in D
                if data[ii, jj] == "."
                    fix(x[ii, jj], 0.0; force = true)
                end
            end

            @objective(
                increased_capacity_model,
                Min,
                sum(data[ii, jj] * x[ii, jj] for ii in O, jj in D if data[ii, jj] isa Number),
            )

            @constraint(increased_capacity_model, [ii in O], sum(x[ii, :]) == data[ii, "EVACUEES"])
            @constraint(increased_capacity_model, [jj in D], sum(x[:, jj]) <= data["VACANCIES", jj])

            optimize!(increased_capacity_model)
            increased_capacity_objective = objective_value(increased_capacity_model)

            # Decrease the capacity slightly and re-solve
            data["VACANCIES", d] = original_capacity - epsilon
            decreased_capacity_model = Model(HiGHS.Optimizer)
            set_silent(decreased_capacity_model)
            @variable(decreased_capacity_model, x[ii in O, jj in D] >= 0)

            for ii in O, jj in D
                if data[ii, jj] == "."
                    fix(x[ii, jj], 0.0; force = true)
                end
            end

            @objective(
                decreased_capacity_model,
                Min,
                sum(data[ii, jj] * x[ii, jj] for ii in O, jj in D if data[ii, jj] isa Number),
            )

            @constraint(decreased_capacity_model, [ii in O], sum(x[ii, :]) == data[ii, "EVACUEES"])
            @constraint(decreased_capacity_model, [jj in D], sum(x[:, jj]) <= data["VACANCIES", jj])

            optimize!(decreased_capacity_model)
            decreased_capacity_objective = objective_value(decreased_capacity_model)

            # Calculate the numerical derivative
            derivative = (increased_capacity_objective - decreased_capacity_objective) / (2 * epsilon)
            derivative_matrix[i, j] = derivative

            # Reset the original capacity
            data["VACANCIES", d] = original_capacity
        end
    end

    return derivative_matrix
end

# Calculate and print the numerical derivative matrix for shelter capacity changes
epsilon = 1000.0
capacity_derivative_matrix = calculate_numerical_capacity_derivative(data, epsilon)
println("\nNumerical Derivative Matrix for Shelter Capacity Changes:")
for row in eachrow(capacity_derivative_matrix)
    println(row)
end

2-dimensional DenseAxisArray{Any,2,...} with index sets:
    Dimension 1, Any["TTWR", "ZUHO", "OSAK", "TDAI", "AERT", "TEPC", "SIPO", "MEDI", "SSTA", "AQUA"  …  "TLDA", "TLMP", "TLTP", "TLMN", "TLHW", "TLMS", "TLH1", "TLH2", "TLH3", "VACANCIES"]
    Dimension 2, Any["広瀬中", "仙台青", "蒲町小", "長町中", "七郷小", "南光台", "新田小", "仙台高", "錦ケ丘", "上杉山"  …  "TSDA", "TSMP", "TSTP", "TSMN", "TSHW", "TSMS", "TSH1", "TSH2", "TSH3", "EVACUEES"]
And data, a 25×28 Matrix{Any}:
      "."       "."       "."      2.85       "."       "."       "."      4.8        "."      1.65      2.31       "."       "."      1.47      4.33       "."      4.38       1.39       4.37       3.94       1.99        "."       3.88       2.46       3.76       4.34       4.49     894
      "."       "."       "."      2.3        "."       "."       "."      4.58       "."      2.36      3.37       "."       "."      2.54      4.12      4.54      3.57       1.2         "."       3.0        3.03        "."       4.03       3.44       3.85

YURT      .      .      .      .      .      .      .      .      .      .      . 11467.0      .      .      .      .      .      .      .      .      .      .      .      .      .      .      .
CAST      .      .      .      .      .      .      .      .      .      .      .      .      .      .      .   87.0      .      .      .      .      .      .      .      .      .      .      .
RAKU      .      . 12950.0      . 12195.0      . 1855.0      .      .      .      .      .      .      .      .      .      .      .      .      .      .      .      .      .      .      .      .
TLSW 15065.0      .      .      .      .      .      .      .      .      .      .      .      .      .      . 14930.0 18518.0 951486.0      .      .      .      .      .      .      .      .      .
TLDA      .      .      .      .      . 13261.0      .      .      .      .      . 11208.0      .      .      .      .      .      . 975530.0      .      .      .      .      .      .      .      .
TLMP      .      .

[-0.058309999999939466, -9.42830999999994, -5.129419999999926, -6.1494199999999255, -3.9594199999999256, -5.44830999999994, -4.8994199999999255, -10.01830999999994, 0.0, -8.82830999999994, -7.839419999999926, -4.098309999999939, -2.1594199999999253, -8.689419999999926, -4.889419999999926, -3.7683099999999397, -4.728309999999939, -9.41830999999994, -7.308309999999939, -5.461789999999921, -8.139419999999925, -4.683309999999939, -5.909419999999925, -7.989419999999925, -5.879419999999926, -5.804419999999926, -5.674419999999926]
[-0.058309999999939466, -9.42830999999994, -5.129419999999926, -6.1494199999999255, -3.9594199999999256, -5.44830999999994, -4.8994199999999255, -10.01830999999994, 0.0, -8.82830999999994, -7.839419999999926, -4.098309999999939, -2.1594199999999253, -8.689419999999926, -4.889419999999926, -3.7683099999999397, -4.728309999999939, -9.41830999999994, -7.308309999999939, -5.461789999999921, -8.139419999999925, -4.683309999999939, -5.909419999999925, -7.989419999999925, 

In [ ]:
using JuMP
import DelimitedFiles
using DelimitedFiles
import HiGHS

open(joinpath(@__DIR__, "transp.txt"), "w") do io
    print(
        io,
        """
             . 広瀬中	仙台青	蒲町小	長町中	七郷小	南光台	新田小	仙台高	錦ケ丘	上杉山	宮城野	仙台商	岩切小	榴岡小	東長町	富沢小	富沢中	TSSW	TSDA	TSMP	TSTP	TSMN	TSHW	TSMS	TSH1	TSH2	TSH3  EVACUEES
          TTWR .	.	.	2.85	.	.	.	4.80	.	1.65	2.31	.	.	1.47	4.33	.	4.38	1.39	4.37	3.94	1.99	.	3.88	2.46	3.76	4.34	4.49	894
          ZUHO .	.	.	2.30	.	.	.	4.58	.	2.36	3.37	.	.	2.54	4.12	4.54	3.57	1.20	.	3.00	3.03	.	4.03	3.44	3.85	4.53	4.64	60
          OSAK .	2.05	.	.	.	.	.	1.46	.	2.65	.	.	.	4.45	.	.	.	2.06	4.17	.	.	.	.	.	.	.	.	100000
          TDAI .	.	.	2.54	.	.	.	4.82	.	1.93	2.61	.	.	1.78	4.11	4.98	4.03	1.35	4.69	3.57	2.27	.	3.77	2.70	3.63	4.25	4.39	5000
          AERT .	.	.	3.56	.	4.54	4.64	4.84	.	1.20	1.77	.	.	0.93	4.86	.	.	1.75	3.70	4.72	1.50	.	4.22	2.06	4.13	4.64	4.81	584
          TEPC .	4.81	.	3.86	.	4.33	4.78	4.47	.	0.78	2.04	.	.	1.25	.	.	.	1.58	3.36	4.92	1.82	.	4.64	2.40	4.54	.	.	12500
          SIPO .	.	.	.	.	.	.	.	.	.	.	3.68	.	.	.	.	.	.	.	.	.	4.29	.	.	.	.	.	1653
          MEDI .	3.88	.	3.89	.	4.99	.	3.45	.	0.99	3.21	.	.	2.39	.	.	.	0.57	3.65	4.54	2.97	.	.	3.53	.	.	.	346
          SSTA .	.	4.83	3.35	.	4.71	4.60	.	.	1.47	1.66	.	.	0.81	4.61	.	4.96	1.87	3.94	4.58	1.36	.	3.95	1.90	3.86	4.37	4.54	75608
          AQUA .	.	.	.	.	.	4.97	.	.	.	.	.	4.19	.	.	.	.	.	.	.	.	.	.	.	.	.	.	913
          MOPS .	.	.	.	.	.	.	.	.	.	.	.	5.09	.	.	.	.	.	.	.	.	.	.	.	.	.	.	10000
          YURT .	.	.	.	.	2.70	.	.	.	.	.	1.03	.	.	.	.	.	.	2.58	.	.	0.49	.	.	.	.	.	11467
          CAST .	4.51	.	2.90	.	.	.	3.87	.	2.68	4.27	.	.	3.42	4.85	4.72	3.77	1.15	.	3.01	3.95	.	4.95	4.41	4.76	.	.	87
          RAKU .	.	2.96	3.98	4.13	4.78	3.19	.	.	3.13	0.68	.	.	1.21	4.39	.	.	3.79	4.74	.	0.67	.	3.24	0.13	3.27	3.48	3.68	27000
          TLSW 9.36	4.01	6.62	3.51	7.86	5.56	6.33	3.49	9.50	1.55	3.51	7.02	8.45	2.66	5.30	5.65	4.69	0.00	4.21	4.02	3.24	6.61	5.10	3.77	4.95	5.59	5.72   999999
          TLDA .	4.38	7.45	7.22	8.38	1.86	4.90	4.57	.	2.80	4.08	3.21	5.62	3.91	8.53	9.66	8.71	4.23	0.00	8.16	4.17	2.66	7.74	4.63	7.69	8.09	8.28   999999
          TLMP .	7.36	7.04	1.70	8.21	9.21	8.68	6.67	9.68	5.35	5.76	.	.	5.11	3.11	1.75	0.89	4.02	8.13	0.00	5.41	.	4.03	5.60	3.80	4.45	4.45   999999
          TLTP .	6.56	3.63	3.96	4.79	4.38	3.37	6.26	.	2.47	0.36	7.35	5.98	0.58	4.66	6.50	5.66	3.23	4.16	5.47	0.00	6.78	3.64	0.58	3.64	3.95	4.14   999999
          TLMN .	5.20	9.87	9.82	.	3.07	6.79	5.70	.	5.35	6.67	0.62	6.40	6.57	.	.	.	6.62	2.66	.	6.80	0.00	.	7.22	.	.	.   999999
          TLHW .	9.02	3.23	2.47	4.29	8.01	5.98	8.50	.	5.38	3.86	.	8.86	3.81	1.38	4.04	3.62	5.05	7.72	4.03	3.64	.	0.00	3.37	0.23	0.50	0.62   999999
          TLMS .	7.13	3.05	4.07	4.20	4.65	3.12	6.83	.	3.05	0.55	7.77	5.86	1.14	4.51	6.56	5.77	3.76	4.61	5.66	0.58	7.19	3.37	0.00	3.40	3.62	3.81   999999
          TLH1 .	8.88	3.43	2.24	4.50	8.02	6.09	8.35	.	5.29	3.87	.	8.96	3.78	1.23	3.84	3.40	4.90	7.68	3.80	3.64	.	0.23	3.40	0.00	0.71	0.79   999999
          TLH2 .	9.49	3.02	2.94	3.99	8.27	6.04	8.98	.	5.80	4.13	.	8.92	4.17	1.59	4.32	3.98	5.53	8.07	4.45	3.95	.	0.50	3.62	0.71	0.00	0.20   999999
          TLH3 .	9.63	3.16	2.97	4.09	8.46	6.23	9.11	.	5.97	4.33	.	9.10	4.35	1.50	4.23	3.94	5.66	8.25	4.44	4.14	.	0.62	3.81	0.79	0.20	0.00   999999
        VACANCIES 15232 14955 12950 16378 12195 13261 21261 10464 14474 14445 14733 24328 16014 13779 12595 15111 18578 999999 999999 999999 999999 999999 999999 999999 999999 999999 999999 0
        """,
    )
    return
end

function read_data(filename::String)
    data = DelimitedFiles.readdlm(filename)
    rows, columns = data[2:end, 1], data[1, 2:end]
    return Containers.DenseAxisArray(data[2:end, 2:end], rows, columns)
end

data = read_data(joinpath(@__DIR__, "transp.txt"))
print(data)

function solve_transportation_problem(data::Containers.DenseAxisArray)
    # Get the set of supplies and demands
    O, D = axes(data)
    # Drop the EVACUEES and VACANCIES nodes from our sets
    O, D = setdiff(O, ["VACANCIES"]), setdiff(D, ["EVACUEES"])
    model = Model(HiGHS.Optimizer)
    set_silent(model)
    @variable(model, x[o in O, d in D] >= 0)
    # Remove arcs with "." cost by fixing them to 0.0.
    for o in O, d in D
        if data[o, d] == "."
            fix(x[o, d], 0.0; force = true)
        end
    end
    @objective(
        model,
        Min,
        sum(data[o, d] * x[o, d] for o in O, d in D if data[o, d] != "."),
    )
    @constraint(model, [o in O], sum(x[o, :]) == data[o, "EVACUEES"])
    @constraint(model, [d in D], sum(x[:, d]) <= data["VACANCIES", d])
    optimize!(model)
    # Pretty print the solution in the format of the input
    print("    ", join(lpad.(D, 7, ' ')))
    for o in O
        print("\n", o)
        for d in D
            if isapprox(value(x[o, d]), 0.0; atol = 1e-6)
                print("      .")
            else
                print(" ", lpad(value(x[o, d]), 6, ' '))
            end
        end
    end
    return
end

print("\nSolution:\n")
solve_transportation_problem(data)

#Monte Carlo
# Import DelimitedFiles module again
using DelimitedFiles

# Define the second read_data function for Monte Carlo simulation
function read_data(filename::String)
    data = readdlm(filename, ' ', Any, '\n')
    header = data[1, 2:end]
    rows = data[2:end, 1]
    values = data[2:end, 2:end]
    data_dict = Dict(zip(header, values))
    data_dict["EVACUEES"] = [parse(Float64, x) for x in data_dict["EVACUEES"]]
    return rows, header, data_dict
end

rows, header, data_dict = read_data(joinpath(@__DIR__, "transp.txt"))

function monte_carlo_simulation(data_dict, population_change::Float64)
    for key in keys(data_dict)
        if key != "EVACUEES" && key != "VACANCIES"
            data_dict[key] = [x * (1.0 + population_change) for x in data_dict[key]]
        end
    end
    
    println("Solution with Population Change: $((1.0 + population_change) * 100)%")
    println("    ", join(lpad.(header, 7, ' ')))
    for row in rows
        print("\n", row)
        for col in header
            if isapprox(data_dict[col][findfirst(x -> x == row, rows)], 0.0; atol = 1e-6)
                print("      .")
            else
                print(" ", lpad(data_dict[col][findfirst(x -> x == row, rows)], 6, ' '))
            end
        end
    end
end

# Define the population change (e.g., 10% increase)
population_change = 0.10

# Run the Monte Carlo simulation
monte_carlo_simulation(data_dict, population_change)


2-dimensional DenseAxisArray{Any,2,...} with index sets:
    Dimension 1, Any["TTWR", "ZUHO", "OSAK", "TDAI", "AERT", "TEPC", "SIPO", "MEDI", "SSTA", "AQUA"  …  "TLDA", "TLMP", "TLTP", "TLMN", "TLHW", "TLMS", "TLH1", "TLH2", "TLH3", "VACANCIES"]
    Dimension 2, Any["広瀬中", "仙台青", "蒲町小", "長町中", "七郷小", "南光台", "新田小", "仙台高", "錦ケ丘", "上杉山"  …  "TSDA", "TSMP", "TSTP", "TSMN", "TSHW", "TSMS", "TSH1", "TSH2", "TSH3", "EVACUEES"]
And data, a 25×28 Matrix{Any}:
      "."       "."       "."      2.85       "."       "."       "."      4.8        "."      1.65      2.31       "."       "."      1.47      4.33       "."      4.38       1.39       4.37       3.94       1.99        "."       3.88       2.46       3.76       4.34       4.49     894
      "."       "."       "."      2.3        "."       "."       "."      4.58       "."      2.36      3.37       "."       "."      2.54      4.12      4.54      3.57       1.2         "."       3.0        3.03        "."       4.03       3.44       3.85

YURT      .      .      .      .      .      .      .      .      .      .      . 11467.0      .      .      .      .      .      .      .      .      .      .      .      .      .      .      .
CAST      .      .      .      .      .      .      .      .      .      .      .      .      .      .      .   87.0      .      .      .      .      .      .      .      .      .      .      .
RAKU      .      . 12950.0      . 12195.0      . 1855.0      .      .      .      .      .      .      .      .      .      .      .      .      .      .      .      .      .      .      .      .
TLSW 15065.0      .      .      .      .      .      .      .      .      .      .      .      .      .      . 14930.0 18518.0 951486.0      .      .      .      .      .      .      .      .      .
TLDA      .      .      .      .      . 13261.0      .      .      .      .      . 11208.0      .      .      .      .      .      . 975530.0      .      .      .      .      .      .      .      .
TLMP      .      .

LoadError: MethodError: no method matching *(::Char, ::Float64)
[0mClosest candidates are:
[0m  *(::Any, ::Any, [91m::Any[39m, [91m::Any...[39m) at /Applications/Julia-1.7.app/Contents/Resources/julia/share/julia/base/operators.jl:655
[0m  *(::Union{AbstractChar, AbstractString}, [91m::Union{AbstractChar, AbstractString}...[39m) at /Applications/Julia-1.7.app/Contents/Resources/julia/share/julia/base/strings/basic.jl:260
[0m  *(::Union{AbstractChar, AbstractString}, [91m::Missing[39m) at /Applications/Julia-1.7.app/Contents/Resources/julia/share/julia/base/missing.jl:183
[0m  ...

In [ ]:
#I get a set of routes from original calculation
#900TTWR -- Shelter 1 100 #Original
#905 TTWR -- Shetler 1 100 #Modified

#Shelter 1 cap: 10,000
#Shelter 2 cap: 10

#Selter 1 cap1: 9500
#Shelter 2 cap1: 9.5


#Optimization may favor certain shelters
    #Step 1: Fit as many as possible based on original simulation
    #Step 2: Count the number of misfits
    #Step 3: Calculate the difference Monte Carlo - Original
    #Step 4: Average out variance

    #Step 3: Run optimization again with misfits and remaining capacities

#I use the same routes/distributions
#I create variance in population, shelter, distance by 0.05, 0.1 0.3
#Calculate total distance for these variances

#If you can't fit in everyone
    #Throw then in to somewhere
    #if they dont fit in anywhere

#See if very dependent on a shelter

#Monte Carlo
using JuMP
import DelimitedFiles
import HiGHS

open(joinpath(@__DIR__, "transp.txt"), "w") do io
    print(
        io,
        """
             . 広瀬中	仙台青	蒲町小	長町中	七郷小	南光台	新田小	仙台高	錦ケ丘	上杉山	宮城野	仙台商	岩切小	榴岡小	東長町	富沢小	富沢中	TSSW	TSDA	TSMP	TSTP	TSMN	TSHW	TSMS	TSH1	TSH2	TSH3  EVACUEES
          TTWR .	.	.	2.85	.	.	.	4.80	.	1.65	2.31	.	.	1.47	4.33	.	4.38	1.39	4.37	3.94	1.99	.	3.88	2.46	3.76	4.34	4.49	894
          ZUHO .	.	.	2.30	.	.	.	4.58	.	2.36	3.37	.	.	2.54	4.12	4.54	3.57	1.20	.	3.00	3.03	.	4.03	3.44	3.85	4.53	4.64	60
          OSAK .	2.05	.	.	.	.	.	1.46	.	2.65	.	.	.	4.45	.	.	.	2.06	4.17	.	.	.	.	.	.	.	.	100000
          TDAI .	.	.	2.54	.	.	.	4.82	.	1.93	2.61	.	.	1.78	4.11	4.98	4.03	1.35	4.69	3.57	2.27	.	3.77	2.70	3.63	4.25	4.39	5000
          AERT .	.	.	3.56	.	4.54	4.64	4.84	.	1.20	1.77	.	.	0.93	4.86	.	.	1.75	3.70	4.72	1.50	.	4.22	2.06	4.13	4.64	4.81	584
          TEPC .	4.81	.	3.86	.	4.33	4.78	4.47	.	0.78	2.04	.	.	1.25	.	.	.	1.58	3.36	4.92	1.82	.	4.64	2.40	4.54	.	.	12500
          SIPO .	.	.	.	.	.	.	.	.	.	.	3.68	.	.	.	.	.	.	.	.	.	4.29	.	.	.	.	.	1653
          MEDI .	3.88	.	3.89	.	4.99	.	3.45	.	0.99	3.21	.	.	2.39	.	.	.	0.57	3.65	4.54	2.97	.	.	3.53	.	.	.	346
          SSTA .	.	4.83	3.35	.	4.71	4.60	.	.	1.47	1.66	.	.	0.81	4.61	.	4.96	1.87	3.94	4.58	1.36	.	3.95	1.90	3.86	4.37	4.54	75608
          AQUA .	.	.	.	.	.	4.97	.	.	.	.	.	4.19	.	.	.	.	.	.	.	.	.	.	.	.	.	.	913
          MOPS .	.	.	.	.	.	.	.	.	.	.	.	5.09	.	.	.	.	.	.	.	.	.	.	.	.	.	.	10000
          YURT .	.	.	.	.	2.70	.	.	.	.	.	1.03	.	.	.	.	.	.	2.58	.	.	0.49	.	.	.	.	.	11467
          CAST .	4.51	.	2.90	.	.	.	3.87	.	2.68	4.27	.	.	3.42	4.85	4.72	3.77	1.15	.	3.01	3.95	.	4.95	4.41	4.76	.	.	87
          RAKU .	.	2.96	3.98	4.13	4.78	3.19	.	.	3.13	0.68	.	.	1.21	4.39	.	.	3.79	4.74	.	0.67	.	3.24	0.13	3.27	3.48	3.68	27000
          TLSW 9.36	4.01	6.62	3.51	7.86	5.56	6.33	3.49	9.50	1.55	3.51	7.02	8.45	2.66	5.30	5.65	4.69	0.00	4.21	4.02	3.24	6.61	5.10	3.77	4.95	5.59	5.72   999999
          TLDA .	4.38	7.45	7.22	8.38	1.86	4.90	4.57	.	2.80	4.08	3.21	5.62	3.91	8.53	9.66	8.71	4.23	0.00	8.16	4.17	2.66	7.74	4.63	7.69	8.09	8.28   999999
          TLMP .	7.36	7.04	1.70	8.21	9.21	8.68	6.67	9.68	5.35	5.76	.	.	5.11	3.11	1.75	0.89	4.02	8.13	0.00	5.41	.	4.03	5.60	3.80	4.45	4.45   999999
          TLTP .	6.56	3.63	3.96	4.79	4.38	3.37	6.26	.	2.47	0.36	7.35	5.98	0.58	4.66	6.50	5.66	3.23	4.16	5.47	0.00	6.78	3.64	0.58	3.64	3.95	4.14   999999
          TLMN .	5.20	9.87	9.82	.	3.07	6.79	5.70	.	5.35	6.67	0.62	6.40	6.57	.	.	.	6.62	2.66	.	6.80	0.00	.	7.22	.	.	.   999999
          TLHW .	9.02	3.23	2.47	4.29	8.01	5.98	8.50	.	5.38	3.86	.	8.86	3.81	1.38	4.04	3.62	5.05	7.72	4.03	3.64	.	0.00	3.37	0.23	0.50	0.62   999999
          TLMS .	7.13	3.05	4.07	4.20	4.65	3.12	6.83	.	3.05	0.55	7.77	5.86	1.14	4.51	6.56	5.77	3.76	4.61	5.66	0.58	7.19	3.37	0.00	3.40	3.62	3.81   999999
          TLH1 .	8.88	3.43	2.24	4.50	8.02	6.09	8.35	.	5.29	3.87	.	8.96	3.78	1.23	3.84	3.40	4.90	7.68	3.80	3.64	.	0.23	3.40	0.00	0.71	0.79   999999
          TLH2 .	9.49	3.02	2.94	3.99	8.27	6.04	8.98	.	5.80	4.13	.	8.92	4.17	1.59	4.32	3.98	5.53	8.07	4.45	3.95	.	0.50	3.62	0.71	0.00	0.20   999999
          TLH3 .	9.63	3.16	2.97	4.09	8.46	6.23	9.11	.	5.97	4.33	.	9.10	4.35	1.50	4.23	3.94	5.66	8.25	4.44	4.14	.	0.62	3.81	0.79	0.20	0.00   999999
        VACANCIES 15232000 14955 12950 16378 12195 13261 21261 10464 14474 14445 14733 24328 16014 13779 12595 15111 18578 999999 999999 999999 999999 999999 999999 999999 999999 999999 999999 0
        """,
    )
    return
end

function read_data(filename::String)
    data = DelimitedFiles.readdlm(filename)
    rows, columns = data[2:end, 1], data[1, 2:end]
    return Containers.DenseAxisArray(data[2:end, 2:end], rows, columns)
end

data = read_data(joinpath(@__DIR__, "transp.txt"))

using Random

# Function to add random variations to durations
function add_duration_variations(data::JuMP.Containers.DenseAxisArray, variation_percentage::Float64)
    O, D = axes(data)
    O, D = setdiff(O, ["VACANCIES"]), setdiff(D, ["EVACUEES"])
    
    for o in O, d in D
        original_cost = data[o, d]
        
        if original_cost isa Number
            variation = original_cost * variation_percentage * (2 * rand() - 1)
            data[o, d] = max(original_cost + variation, 0.0)
        end
    end
end

# Function to add random variations to capacities
function add_capacity_variations(data::JuMP.Containers.DenseAxisArray, variation_percentage::Float64)
    D = axes(data)[2]
    D = setdiff(D, ["EVACUEES"])
    
    for d in D
        original_capacity = data["VACANCIES", d]
        
        if original_capacity isa Number
            variation = original_capacity * variation_percentage * (2 * rand() - 1)
            data["VACANCIES", d] = max(original_capacity + variation, 0.0)
        end
    end
end

# Function to perform a single trial of the Monte Carlo simulation
function run_monte_carlo_trial(data::JuMP.Containers.DenseAxisArray, duration_variation::Float64, capacity_variation::Float64)
    data_copy = deepcopy(data)
    add_duration_variations(data_copy, duration_variation)
    add_capacity_variations(data_copy, capacity_variation)
    println("\nTrial with Duration Variation: $duration_variation, Capacity Variation: $capacity_variation")
    
    # Attempt to solve the transportation problem
    status = solve_transportation_problem(data_copy)
    
    # Check if the solver found a valid solution
    if status == MOI.OPTIMAL
        # Access and print results
        println("Objective Value: ", JuMP.objective_value(data_copy.model))
        println("Transportation Plan:")
        for o in O, d in D
            if haskey(data_copy, ("VACANCIES", d), (o, d))
                println("$o -> $d: ", value(data_copy[("VACANCIES", d), (o, d)]))
            end
        end
    else
        println("Solver couldn't find a valid solution.")
    end
end

# Number of Monte Carlo iterations
num_iterations = 100

# Range of percentage variations for durations and capacities
duration_variations = [0.05]  # You can add more variation levels
capacity_variations = [0.05]  # You can add more variation levels

# Perform Monte Carlo simulation
for duration_variation in duration_variations
    for capacity_variation in capacity_variations
        for i in 1:num_iterations
            run_monte_carlo_trial(data, duration_variation, capacity_variation)
        end
    end
end



Trial with Duration Variation: 0.05, Capacity Variation: 0.05


LoadError: UndefVarError: solve_transportation_problem not defined

In [ ]:
#no transshipment nodes
using JuMP
import DelimitedFiles
import HiGHS

open(joinpath(@__DIR__, "transp.txt"), "w") do io
    print(
        io,
        """
             . 広瀬中   仙台青   蒲町小  長町中  七郷小      新田小  仙台高   南光台   錦ケ丘   上杉山   宮城野   仙台商   岩切小    榴岡小  東長町    富沢小  富沢中  EVACUEES
          TTWR 10.71	5.25	5.23	2.85	6.46	5.29	4.80	5.31	10.79	1.65	2.31	7.46	7.68	1.47	4.33	5.32	4.38  1178
          ZUHO 9.99 	5.15	5.96	2.30	7.21	6.35	4.58	6.27	9.92	2.36	3.37	8.10	8.76	2.54	4.12	4.54	3.57  60
          OSAK 7.65 	2.05	8.60	5.35	9.81	7.72	1.46	5.91	8.07	2.65	5.24	6.18	9.36	4.45	7.26	7.13	6.21  100000
          TDAI 10.57	5.32	5.34	2.54	6.59	5.60	4.82	5.66	10.59	1.93	2.61	7.75	8.03	1.78	4.11	4.98	4.03  5000
          AERT 11.05	5.20	5.04	3.56	6.24	4.64	4.84	4.54	11.26	1.20	1.77	6.85	6.94	0.93	4.86	6.07	5.13  773
          TEPC 10.77	4.81	5.43	3.86	6.61	4.78	4.47	4.33	11.03	0.78	2.04	6.49	6.95	1.25	5.24	6.33	5.39  12500
          SIPO 10.21	5.89	13.99	12.75	14.91	11.12	6.61	7.37	11.83	8.49	10.52	3.68	10.65	10.19	14.46	14.79	13.85 1653
          MEDI 9.60 	3.88	6.50	3.89	7.71	5.92	3.45	4.99	9.85	0.99	3.21	6.51	7.95	2.39	5.59	6.14	5.17  346
          SSTA 11.20	5.44	4.83	3.35	6.04	4.60	5.06	4.71	11.37	1.47	1.66	7.10	6.99	0.81	4.61	5.88	4.96  75608
          AQUA 20.33	13.61	6.51	11.64	5.73	4.97	13.62	7.72	20.85	10.09	7.88	10.96	4.19	8.75	11.16	13.84	13.25 913
          MOPS 21.04	14.37	6.77	12.06	5.86	5.67	14.36	8.57	21.52	10.78	8.48	11.84	5.09	9.35	11.45	14.18	13.63 10000
          YURT 12.14	5.55	9.58	9.79	10.35	6.41	6.02	2.70	13.27	5.35	6.47	1.03	5.92	6.43	11.09	12.22	11.26 11467
          CAST 8.98 	4.51	6.99	2.90	8.24	7.21	3.87	6.69	8.90	2.68	4.27	8.07	9.47	3.42	4.85	4.72	3.77  87
          RAKU 13.11	7.21	2.96	3.98	4.13	3.19	6.90	4.78	13.26	3.13	0.68	7.90	5.95	1.21	4.39	6.46	5.68  27000
        VACANCIES 15232 14955 12950 16378 12195 13261 21261 10464 14474 14445 14733 24328 16014 13779 12595 15111 18578 0
        """,
    )
    return
end

function read_data(filename::String)
    data = DelimitedFiles.readdlm(filename)
    rows, columns = data[2:end, 1], data[1, 2:end]
    return Containers.DenseAxisArray(data[2:end, 2:end], rows, columns)
end

data = read_data(joinpath(@__DIR__, "transp.txt"))
print(data)

function solve_transportation_problem(data::Containers.DenseAxisArray)
    # Get the set of supplies and demands
    O, D = axes(data)
    # Drop the EVACUEES and VACANCIES nodes from our sets
    O, D = setdiff(O, ["VACANCIES"]), setdiff(D, ["EVACUEES"])
    model = Model(HiGHS.Optimizer)
    set_silent(model)
    @variable(model, x[o in O, d in D] >= 0)
    # Remove arcs with "." cost by fixing them to 0.0.
    for o in O, d in D
        if data[o, d] == "."
            fix(x[o, d], 0.0; force = true)
        end
    end
    @objective(
        model,
        Min,
        sum(data[o, d] * x[o, d] for o in O, d in D if data[o, d] != "."),
    )
    @constraint(model, [o in O], sum(x[o, :]) == data[o, "EVACUEES"])
    @constraint(model, [d in D], sum(x[:, d]) <= data["VACANCIES", d])
    optimize!(model)
    # Pretty print the solution in the format of the input
    print("    ", join(lpad.(D, 7, ' ')))
    for o in O
        print("\n", o)
        for d in D
            if isapprox(value(x[o, d]), 0.0; atol = 1e-6)
                print("      .")
            else
                print(" ", lpad(value(x[o, d]), 6, ' '))
            end
        end
    end
    return
end

print("\nSolution:\n")
solve_transportation_problem(data)

#########################################

function calculate_numerical_derivative(data::JuMP.Containers.DenseAxisArray{Any, 2}, epsilon::Float64 = 1)
    O, D = axes(data)
    O, D = setdiff(O, ["VACANCIES"]), setdiff(D, ["EVACUEES"])
    num_rows, num_cols = length(O), length(D)
    derivative_matrix = zeros(Float64, num_rows, num_cols)

    for (i, o) in enumerate(O), (j, d) in enumerate(D)
        original_cost = data[o, d]

        if original_cost isa Number
            # Increase the cost slightly and re-solve
            data[o, d] = original_cost + epsilon
            increased_cost_model = Model(HiGHS.Optimizer)
            set_silent(increased_cost_model)
            @variable(increased_cost_model, x[ii in O, jj in D] >= 0)

            for ii in O, jj in D
                if data[ii, jj] == "."
                    fix(x[ii, jj], 0.0; force = true)
                end
            end

            @objective(
                increased_cost_model,
                Min,
                sum(data[ii, jj] * x[ii, jj] for ii in O, jj in D if data[ii, jj] isa Number),
            )

            @constraint(increased_cost_model, [ii in O], sum(x[ii, :]) == data[ii, "EVACUEES"])
            @constraint(increased_cost_model, [jj in D], sum(x[:, jj]) <= data["VACANCIES", jj])

            optimize!(increased_cost_model)
            increased_cost_objective = objective_value(increased_cost_model)

            # Decrease the cost slightly and re-solve
            data[o, d] = original_cost - epsilon
            decreased_cost_model = Model(HiGHS.Optimizer)
            set_silent(decreased_cost_model)
            @variable(decreased_cost_model, x[ii in O, jj in D] >= 0)

            for ii in O, jj in D
                if data[ii, jj] == "."
                    fix(x[ii, jj], 0.0; force = true)
                end
            end

            @objective(
                decreased_cost_model,
                Min,
                sum(data[ii, jj] * x[ii, jj] for ii in O, jj in D if data[ii, jj] isa Number),
            )

            @constraint(decreased_cost_model, [ii in O], sum(x[ii, :]) == data[ii, "EVACUEES"])
            @constraint(decreased_cost_model, [jj in D], sum(x[:, jj]) <= data["VACANCIES", jj])

            optimize!(decreased_cost_model)
            decreased_cost_objective = objective_value(decreased_cost_model)

            # Calculate the numerical derivative
            derivative = (increased_cost_objective - decreased_cost_objective) / (2 * epsilon)
            derivative_matrix[i, j] = derivative

            # Reset the original cost
            data[o, d] = original_cost
        end
    end

    return derivative_matrix
end

# Calculate and print the numerical derivative matrix
epsilon = 1.0
derivative_matrix = calculate_numerical_derivative(data, epsilon)
println("\nNumerical Derivative Matrix:")
for row in eachrow(derivative_matrix)
    println(row)
end

2-dimensional DenseAxisArray{Any,2,...} with index sets:
    Dimension 1, Any["TTWR", "ZUHO", "OSAK", "TDAI", "AERT", "TEPC", "SIPO", "MEDI", "SSTA", "AQUA", "MOPS", "YURT", "CAST", "RAKU", "VACANCIES"]
    Dimension 2, Any["広瀬中", "仙台青", "蒲町小", "長町中", "七郷小", "新田小", "仙台高", "南光台", "錦ケ丘", "上杉山", "宮城野", "仙台商", "岩切小", "榴岡小", "東長町", "富沢小", "富沢中", "EVACUEES"]
And data, a 15×18 Matrix{Any}:
    10.71      5.25      5.23      2.85      6.46      5.29      4.8       5.31     10.79      1.65      2.31      7.46      7.68      1.47      4.33      5.32      4.38    1178
     9.99      5.15      5.96      2.3       7.21      6.35      4.58      6.27      9.92      2.36      3.37      8.1       8.76      2.54      4.12      4.54      3.57      60
     7.65      2.05      8.6       5.35      9.81      7.72      1.46      5.91      8.07      2.65      5.24      6.18      9.36      4.45      7.26      7.13      6.21  100000
    10.57      5.32      5.34      2.54      6.59      5.6       4.82      5.66 

In [ ]:
#Mod-Haversine Euclidean DISTANCE CALCULATOR 1
using Distances

function distance(lat, lng, lat0, lng0)
    deglen = 110.25
    x = lat - lat0
    y = (lng - lng0) * cos(lat0)
    return deglen * sqrt(x*x + y*y)
end

#Data
locations_dict = Dict("Zuihōden Temple" => (38.25076685689052, 140.8667686177859),
"Sendai Castle" => (38.25209, 140.85592),
"Sendai Station" => (38.26028322570647, 140.88239995371939),
"Rakuten Seimei Park Miyagi" => (38.25695841274225, 140.90267074199306),
"AER" => (38.26243, 140.88104),
"Osaki Hachiman Shrine" => (38.27251, 140.84503),
"Sendai Umino-Mori Aquarium" => (38.280300194102374, 140.98200323180546),
"Sendai Trust Tower" => (38.25655, 140.87626),
"Yurtec Stadium Sendai" => (38.31933922945424, 140.88186037724955),
"Sendai Izumi PREMIUM OUTLETS" => (38.34201389944614, 140.83717960908587),
"Sendai Mediatheque" => (38.26536, 140.86568),
"Mitsui Outlet Park Sendai Port" => (38.27518, 140.98983),
"Tohoku University Katahira Campus" => (38.25383, 140.87407),
"Tohoku Electric Power Company" => (38.26555, 140.87835),
"Transshipment-location Dainoharashinrinkoen" => (38.29598, 140.88088),
"Transshipment-location Sendai West Park" => (38.26098, 140.86235),
"Transshipment-location Miyaginohara Sports Park" => (38.25816, 140.90253),
"Transshipment-location Tsutsujigaoka Park" => (38.26067, 140.89702),
"Transshipment-location Hirosegawa Wakabayashi Ryokuchi Park" => (38.22769, 140.89955),
"Transshipment-location Hirosegawanakagawara Ryokuchi Park" => (38.22538, 140.90411),
"Transshipment-location Hirosegawahachihonmatsu Ryokuchi Park" => (38.22766, 140.89715),
"Transshipment-location Hirose River Iida Green Space" => (38.22359, 140.90396),
"Transshipment-location Mikamine Park" => (38.2248, 140.8574),
"Transshipment-location Moomin Nanakita Park" => (38.31981, 140.87641))

shelters_dict = Dict("錦ケ丘小" => (38.25644, 140.76093),
"上杉山通小" => (38.27135, 140.87361),
"広瀬中" => (38.27389, 140.76242),
"仙台高" => (38.2821, 140.83406),
"仙台青陵中等教育" => (38.28862, 140.83386),
"榴岡小" => (38.26151, 140.89091),
"岩切小" => (38.29979, 140.94251),
"新田小" => (38.27628, 140.92843),
"宮城野中" => (38.26269, 140.90001),
"七郷小" => (38.24103, 140.94222),
"蒲町小" => (38.24245, 140.92912),
"東長町小" => (38.21892, 140.8892),
"富沢小" => (38.20972, 140.86297),
"長町中" => (38.23077, 140.87384),
"富沢中" => (38.21849, 140.86325),
"南光台小" => (38.30032, 140.9006),
"仙台商業高" => (38.32415, 140.87201),
"Transshipment-shelter Dainoharashinrinkoen" => (38.29598, 140.88088),
"Transshipment-shelter Sendai West Park" => (38.26098, 140.86235),
"Transshipment-shelter Miyaginohara Sports Park" => (38.25816, 140.90253),
"Transshipment-shelter Tsutsujigaoka Park" => (38.26067, 140.89702),
"Transshipment-shelter Hirosegawa Wakabayashi Ryokuchi Park" => (38.22769, 140.89955),
"Transshipment-shelter Hirosegawanakagawara Ryokuchi Park" => (38.22538, 140.90411),
"Transshipment-shelter Hirosegawahachihonmatsu Ryokuchi Park" => (38.22766, 140.89715),
"Transshipment-shelter Hirose River Iida Green Space" => (38.22359, 140.90396),
"Transshipment-shelter Mikamine Park" => (38.2248, 140.8574),
"Transshipment-shelter Moomin Nanakita Park" => (38.31981, 140.87641))

#Calculate
distances_dict = Dict{String, Dict{String, Float64}}()

for (loc_name, loc_coords) in locations_dict
    distances_dict[loc_name] = Dict{String, Float64}()
    
    for (sh_name, sh_coords) in shelters_dict
        distances_dict[loc_name][sh_name] = distance(loc_coords[1], loc_coords[2], sh_coords[1], sh_coords[2])
    end
end

#Print
for (loc_name, distances) in distances_dict
    println("Distances from $loc_name:")
    
    for (sh_name, distance) in distances
        println("  $sh_name: $distance km")
    end
    
    println()
end

Distances from Sendai Trust Tower:
  Transshipment-shelter Sendai West Park: 1.3866670908535144 km
  広瀬中: 10.706165212457629 km
  仙台青陵中等教育: 5.253488264906729 km
  Transshipment-shelter Hirosegawanakagawara Ryokuchi Park: 4.342632564801423 km
  Transshipment-shelter Hirosegawahachihonmatsu Ryokuchi Park: 3.755061197714097 km
  蒲町小: 5.225136877733076 km
  長町中: 2.851534005846954 km
  七郷小: 6.460850045301224 km
  南光台小: 5.308855294814546 km
  新田小: 5.288125807409881 km
  仙台高: 4.797987507462382 km
  Transshipment-shelter Dainoharashinrinkoen: 4.3675234947596095 km
  錦ケ丘小: 10.790975241507619 km
  上杉山通小: 1.6500828140074453 km
  宮城野中: 2.3146910899436053 km
  Transshipment-shelter Mikamine Park: 3.9354660649197175 km
  仙台商業高: 7.462580016934648 km
  岩切小: 7.683293427436732 km
  Transshipment-shelter Moomin Nanakita Park: 6.9744279744032305 km
  榴岡小: 1.4717555808435177 km
  Transshipment-shelter Tsutsujigaoka Park: 1.9898307396072181 km
  東長町小: 4.329538796042483 km
  富沢小: 5.318907136556792 km
  Trans

  Transshipment-shelter Dainoharashinrinkoen: 3.362830527248163 km
  錦ケ丘小: 11.032335498875323 km
  上杉山通小: 0.775828198219253 km
  宮城野中: 2.0431833164628483 km
  Transshipment-shelter Mikamine Park: 4.916883321205494 km
  仙台商業高: 6.485468437239356 km
  岩切小: 6.949993908562245 km
  Transshipment-shelter Moomin Nanakita Park: 5.984694684273462 km
  榴岡小: 1.253279788060108 km
  Transshipment-shelter Tsutsujigaoka Park: 1.8234373138695448 km
  東長町小: 5.244742543621109 km
  富沢小: 6.330537446525945 km
  Transshipment-shelter Hirosegawa Wakabayashi Ryokuchi Park: 4.636423400145812 km
  Transshipment-shelter Miyaginohara Sports Park: 2.4023744179947317 km
  Transshipment-shelter Hirose River Iida Green Space: 5.231993792905879 km
  富沢中: 5.38589301920371 km

Distances from Transshipment-location Mikamine Park:
  Transshipment-shelter Sendai West Park: 4.015492175053694 km
  広瀬中: 10.321648547165989 km
  仙台青陵中等教育: 7.359426071829252 km
  Transshipment-shelter Hirosegawanakagawara Ryokuchi Park: 4.45340800

  Transshipment-shelter Miyaginohara Sports Park: 8.372529813838067 km
  Transshipment-shelter Hirose River Iida Green Space: 9.975128771654358 km
  富沢中: 13.632290939983735 km

Distances from Yurtec Stadium Sendai:
  Transshipment-shelter Sendai West Park: 6.686649979838516 km
  広瀬中: 12.135147151546489 km
  仙台青陵中等教育: 5.551576361430548 km
  Transshipment-shelter Hirosegawanakagawara Ryokuchi Park: 10.573931971984521 km
  Transshipment-shelter Hirosegawahachihonmatsu Ryokuchi Park: 10.211915205281235 km
  蒲町小: 9.578721284848323 km
  長町中: 9.794458969827422 km
  七郷小: 10.346126478852183 km
  南光台小: 2.7017940868199894 km
  新田小: 6.406953211520052 km
  仙台高: 6.017621606982947 km
  Transshipment-shelter Dainoharashinrinkoen: 2.5769062257056126 km
  錦ケ丘小: 13.270940272971876 km
  上杉山通小: 5.345788759660722 km
  宮城野中: 6.470590874086619 km
  Transshipment-shelter Mikamine Park: 10.68078108887895 km
  仙台商業高: 1.0280646643456617 km
  岩切小: 5.9222615074808465 km
  Transshipment-shelter Moomin Nanakita Park:

  蒲町小: 3.63484919677622 km
  長町中: 3.9647512195412897 km
  七郷小: 4.787010230620376 km
  南光台小: 4.383513272469631 km
  新田小: 3.3739314006778036 km
  仙台高: 6.257923861261894 km
  Transshipment-shelter Dainoharashinrinkoen: 4.161849726341583 km
  錦ケ丘小: 12.74193613369052 km
  上杉山通小: 2.468683286650202 km
  宮城野中: 0.3567252739669979 km
  Transshipment-shelter Mikamine Park: 5.469468383094419 km
  仙台商業高: 7.347196723247335 km
  岩切小: 5.976584120881784 km
  Transshipment-shelter Moomin Nanakita Park: 6.777127048848962 km
  榴岡小: 0.5773505519745514 km
  Transshipment-shelter Tsutsujigaoka Park: 0.0 km
  東長町小: 4.663362715865789 km
  富沢小: 6.502319001938733 km
  Transshipment-shelter Hirosegawa Wakabayashi Ryokuchi Park: 3.644014171026461 km
  Transshipment-shelter Miyaginohara Sports Park: 0.5846351826749574 km
  Transshipment-shelter Hirose River Iida Green Space: 4.141369813217257 km
  富沢中: 5.663267097028199 km

Distances from Transshipment-location Hirose River Iida Green Space:
  Transshipment-shelter

In [ ]:
#Mod-Haversine Euclidean DISTANCE CALCULATOR 2
using Distances

function distance(lat, lng, lat0, lng0)
    deglen = 110.25
    x = lat - lat0
    y = (lng - lng0) * cos(lat0)
    return deglen * sqrt(x*x + y*y)
end

#Data
locations_dict = Dict("Zuihōden Temple" => (38.25076685689052, 140.8667686177859),
"Sendai Castle" => (38.25209, 140.85592),
"Sendai Station" => (38.26028322570647, 140.88239995371939),
"Rakuten Seimei Park Miyagi" => (38.25695841274225, 140.90267074199306),
"AER" => (38.26243, 140.88104),
"Osaki Hachiman Shrine" => (38.27251, 140.84503),
"Sendai Umino-Mori Aquarium" => (38.280300194102374, 140.98200323180546),
"Sendai Trust Tower" => (38.25655, 140.87626),
"Yurtec Stadium Sendai" => (38.31933922945424, 140.88186037724955),
"Sendai Izumi PREMIUM OUTLETS" => (38.34201389944614, 140.83717960908587),
"Sendai Mediatheque" => (38.26536, 140.86568),
"Mitsui Outlet Park Sendai Port" => (38.27518, 140.98983),
"Tohoku University Katahira Campus" => (38.25383, 140.87407),
"Tohoku Electric Power Company" => (38.26555, 140.87835),
"Transshipment-location Dainoharashinrinkoen" => (38.29598, 140.88088),
"Transshipment-location Sendai West Park" => (38.26098, 140.86235),
"Transshipment-location Miyaginohara Sports Park" => (38.25816, 140.90253),
"Transshipment-location Tsutsujigaoka Park" => (38.26067, 140.89702),
"Transshipment-location Hirosegawa Wakabayashi Ryokuchi Park" => (38.22769, 140.89955),
"Transshipment-location Hirosegawanakagawara Ryokuchi Park" => (38.22538, 140.90411),
"Transshipment-location Hirosegawahachihonmatsu Ryokuchi Park" => (38.22766, 140.89715),
"Transshipment-location Hirose River Iida Green Space" => (38.22359, 140.90396),
"Transshipment-location Mikamine Park" => (38.2248, 140.8574),
"Transshipment-location Moomin Nanakita Park" => (38.31981, 140.87641))

shelters_dict = Dict("錦ケ丘小" => (38.25644, 140.76093),
"上杉山通小" => (38.27135, 140.87361),
"広瀬中" => (38.27389, 140.76242),
"仙台高" => (38.2821, 140.83406),
"仙台青陵中等教育" => (38.28862, 140.83386),
"榴岡小" => (38.26151, 140.89091),
"岩切小" => (38.29979, 140.94251),
"新田小" => (38.27628, 140.92843),
"宮城野中" => (38.26269, 140.90001),
"七郷小" => (38.24103, 140.94222),
"蒲町小" => (38.24245, 140.92912),
"東長町小" => (38.21892, 140.8892),
"富沢小" => (38.20972, 140.86297),
"長町中" => (38.23077, 140.87384),
"富沢中" => (38.21849, 140.86325),
"南光台小" => (38.30032, 140.9006),
"仙台商業高" => (38.32415, 140.87201),
"Transshipment-shelter Dainoharashinrinkoen" => (38.29598, 140.88088),
"Transshipment-shelter Sendai West Park" => (38.26098, 140.86235),
"Transshipment-shelter Miyaginohara Sports Park" => (38.25816, 140.90253),
"Transshipment-shelter Tsutsujigaoka Park" => (38.26067, 140.89702),
"Transshipment-shelter Hirosegawa Wakabayashi Ryokuchi Park" => (38.22769, 140.89955),
"Transshipment-shelter Hirosegawanakagawara Ryokuchi Park" => (38.22538, 140.90411),
"Transshipment-shelter Hirosegawahachihonmatsu Ryokuchi Park" => (38.22766, 140.89715),
"Transshipment-shelter Hirose River Iida Green Space" => (38.22359, 140.90396),
"Transshipment-shelter Mikamine Park" => (38.2248, 140.8574),
"Transshipment-shelter Moomin Nanakita Park" => (38.31981, 140.87641))

# Calculate and store distances in a nested dictionary
distances_dict = Dict{String, Dict{String, Float64}}()

for (sh_name, sh_coords) in shelters_dict
    distances_dict[sh_name] = Dict{String, Float64}()
    
    for (loc_name, loc_coords) in locations_dict
        distances_dict[sh_name][loc_name] = distance(sh_coords[1], sh_coords[2], loc_coords[1], loc_coords[2])
    end
end

# Print the calculated distances grouped by shelter
for (sh_name, distances) in distances_dict
    println("Distances from $sh_name:")
    
    for (loc_name, distance) in distances
        println("  $loc_name: $distance km")
    end
    
    println()
end


Distances from Transshipment-shelter Sendai West Park:
  Sendai Trust Tower: 1.3900432911278033 km
  Transshipment-location Dainoharashinrinkoen: 4.212489669577508 km
  Zuihōden Temple: 1.2000022924569043 km
  Osaki Hachiman Shrine: 2.0467339029135827 km
  Transshipment-location Miyaginohara Sports Park: 3.768294459183986 km
  Tohoku University Katahira Campus: 1.3519696730601776 km
  AER: 1.7495081791019689 km
  Tohoku Electric Power Company: 1.5714556919781841 km
  Transshipment-location Mikamine Park: 4.016679840373975 km
  Sendai Izumi PREMIUM OUTLETS: 9.205926648198131 km
  Sendai Mediatheque: 0.5737442088516905 km
  Sendai Station: 1.87306292669884 km
  Sendai Umino-Mori Aquarium: 11.229687997291052 km
  Transshipment-location Sendai West Park: 0.0 km
  Mitsui Outlet Park Sendai Port: 11.889918694773973 km
  Yurtec Stadium Sendai: 6.667949566695484 km
  Transshipment-location Hirosegawa Wakabayashi Ryokuchi Park: 5.100322234234599 km
  Sendai Castle: 1.1508937863314395 km
  Trans

  Transshipment-location Miyaginohara Sports Park: 4.16271998009754 km
  Tohoku University Katahira Campus: 6.540921241934849 km
  AER: 6.171673749249518 km
  Tohoku Electric Power Company: 6.527938109733341 km
  Transshipment-location Mikamine Park: 8.284319925259895 km
  Sendai Izumi PREMIUM OUTLETS: 14.486666110351367 km
  Sendai Mediatheque: 7.609872739435999 km
  Sendai Station: 5.9735377623196015 km
  Sendai Umino-Mori Aquarium: 5.673103483193175 km
  Transshipment-location Sendai West Park: 7.769716396930641 km
  Mitsui Outlet Park Sendai Port: 5.792406597208983 km
  Yurtec Stadium Sendai: 10.19133278608103 km
  Transshipment-location Hirosegawa Wakabayashi Ryokuchi Park: 4.320377143153531 km
  Sendai Castle: 8.187866819926143 km
  Transshipment-location Hirosegawahachihonmatsu Ryokuchi Park: 4.53703073443607 km
  Transshipment-location Hirosegawanakagawara Ryokuchi Park: 4.021993276125499 km
  Rakuten Seimei Park Miyagi: 4.094937690723615 km
  Transshipment-location Moomin Nana

  Transshipment-location Sendai West Park: 3.5187436860264643 km
  Mitsui Outlet Park Sendai Port: 8.417867316332858 km
  Yurtec Stadium Sendai: 6.454348125018127 km
  Transshipment-location Hirosegawa Wakabayashi Ryokuchi Park: 3.85899850323957 km
  Sendai Castle: 4.298387059908913 km
  Transshipment-location Hirosegawahachihonmatsu Ryokuchi Park: 3.8716441502641255 km
  Transshipment-location Hirosegawanakagawara Ryokuchi Park: 4.131955671356867 km
  Rakuten Seimei Park Miyagi: 0.6791506653777734 km
  Transshipment-location Moomin Nanakita Park: 6.643654221437625 km
  Transshipment-location Tsutsujigaoka Park: 0.3570027858096953 km
  Transshipment-location Hirose River Iida Green Space: 4.327224701670432 km

Distances from Transshipment-shelter Mikamine Park:
  Sendai Trust Tower: 3.920031004316817 km
  Transshipment-location Dainoharashinrinkoen: 8.134433508027914 km
  Zuihōden Temple: 2.9949466401755704 km
  Osaki Hachiman Shrine: 5.383349400684364 km
  Transshipment-location Miyag

  Transshipment-location Hirosegawahachihonmatsu Ryokuchi Park: 1.2253026531884001 km
  Transshipment-location Hirosegawanakagawara Ryokuchi Park: 1.5898492071932449 km
  Rakuten Seimei Park Miyagi: 4.378927245302598 km
  Transshipment-location Moomin Nanakita Park: 11.182112357673967 km
  Transshipment-location Tsutsujigaoka Park: 4.660425925233524 km
  Transshipment-location Hirose River Iida Green Space: 1.4997084363696678 km

Distances from 富沢小:
  Sendai Trust Tower: 5.310622147742059 km
  Transshipment-location Dainoharashinrinkoen: 9.64937781033672 km
  Zuihōden Temple: 4.539449852794495 km
  Osaki Hachiman Shrine: 7.119205976903372 km
  Transshipment-location Miyaginohara Sports Park: 6.495578504604057 km
  Tohoku University Katahira Campus: 4.973144667269801 km
  AER: 6.050465709316069 km
  Tohoku Electric Power Company: 6.319369215752202 km
  Transshipment-location Mikamine Park: 1.745361859027281 km
  Sendai Izumi PREMIUM OUTLETS: 14.761874174846113 km
  Sendai Mediatheque: 6